In [ ]:
import copy
import itertools
import random
import math
import networkx as nx
import pandas as pd
import numpy as np
import time
import pickle
from collections import Counter
import matplotlib.pyplot as plt
from collections import defaultdict
from functools import reduce
import gc
path='wiki'
with open("./dataset/"+path+'/'+"Ir.pkl", "rb") as f:
    Ir=pickle.load(f)
with open("./dataset/"+path+'/'+"If.pkl", "rb") as f:
    If=pickle.load(f)
with open("./dataset/"+path+'/'+"sorted_list.pkl", "rb") as f:
    sorted_list=pickle.load(f)
with open("./dataset/"+path+'/'+"sh.pkl", "rb") as f:
    sh=pickle.load(f)
with open("./dataset/"+path+'/'+"G.pkl", "rb") as f:
    G=pickle.load(f)   
with open("./dataset/"+path+'/'+"community_list.pkl", "rb") as f:
    community_list=pickle.load(f)
node_list=list(G.nodes)
total_nodes=len(G.nodes)#总节点数
total_edges=len(G.edges)#总边数
nodenum=sorted(node_list,reverse=True)[0]+1
print('len(community_list)',len(community_list))
print('total_nodes:',total_nodes, 'total_edges:',total_edges)
def RRS(g=G,t=node_list):# rr-seed
    seed = random.choice(t)
    result_list = []
    result_list.append(seed)
    RRS=[]
    RRS.append(seed)
    # 保存激活的状态
    checked = np.zeros(nodenum,dtype=bool) 
    for node in result_list:
        checked[node] = 1
    # 当前节点不为空，进行影响
    while len(result_list) != 0:
        current_node = result_list.pop(0)
        for nbr in g.predecessors(current_node): # 得到当前节点的邻居节点
            if checked[nbr] == 0:
                wt = g.get_edge_data(nbr,current_node)
                if random.uniform(0,1) < wt['weight'] :
                    result_list.append(nbr)
                    checked[nbr] = 1
                    RRS.append(nbr)
    return RRS
def update_weight(g,F):
    def calculate_h_uv(Fv, F, b_uv, q_uv):# 计算 h_uv(Fv, F)
        overlap = len(set(Fv) & set(F))  # 计算 Fv 和 F 的交集大小
        if(q_uv==0): max_increase=0
        else:max_increase = (1 - b_uv) / q_uv  # 根据公式1限制激活概率的上限
        return min(max_increase, overlap)
    G=g.copy()
    # 更新图中每条边的激活概率
    for u, v, data in G.edges(data=True):
        b_uv = data['b_uv']  # 获取基础激活概率
        q_uv = data['q_uv']  # 获取边的增益系数
        F_v = G.nodes[v]['community']  # 获取目标节点 v 的社区集合
        # 根据公式计算新的激活概率
        h_uv_value = calculate_h_uv(F_v, F, b_uv, q_uv)
        new_p_uv = b_uv + q_uv * h_uv_value
        # 更新边的激活概率
        if(new_p_uv!=G[u][v]['weight']):
            G[u][v]['weight'] = new_p_uv
    return G
def RRS_ori(g=G,t=node_list):#最原始的rr-seed 
    seed = random.choice(t)
    result_list = []
    result_list.append(seed)
    RRS=[]
    RRS.append(seed)
    # 保存激活的状态
    checked = np.zeros(nodenum,dtype=bool) 
    for node in result_list:
        checked[node] = 1
    # 当前节点不为空，进行影响
    while len(result_list) != 0:
        current_node = result_list.pop(0)
        for nbr in g.predecessors(current_node): # 得到当前节点的邻居节点
            if checked[nbr] == 0:
                wt = g.get_edge_data(nbr,current_node)
                if random.uniform(0,1) < wt['weight'] :
                    result_list.append(nbr)
                    checked[nbr] = 1
                    RRS.append(nbr)
    return RRS
def combination(n, k):
    if k < 0 or k > n:
        return 0
    # 利用对称性，C(n, k) == C(n, n-k)
    if k > n - k:
        k = n - k
    result = 1
    for i in range(1, k + 1):
        result = result * (n - i + 1) // i
    return result
def lnc(n,k):
    return math.log(combination(n, k))
def nodeselect(RR,k):#对RRSet作种子选择
    s=time.time()
    R=RR.copy()
    js=0
    SEED=[]
    for _ in range(k):
        flat_map = [item for subset in R for item in subset]
        if(len(flat_map))==0:
            return SEED,0
        seed = Counter(flat_map).most_common()[0][0]
        js=js+Counter(flat_map).most_common()[0][1]
        SEED.append(seed)
        R = [rrs for rrs in R if seed not in rrs]
    #print('nodeslect:',time.time()-s)
    #print("seed_st:",SEED)
    return SEED,js
def IMM_RRS(G,k,eps,ell,node_list=node_list,n=total_nodes):
    s = time.time()
    # 参数计算（保持不变）
    ell=ell*(1+math.log(2)/math.log(n))
    eps1=math.sqrt(2) * eps
    alpha = math.sqrt(ell * math.log(n) + math.log(2))
    beta = math.sqrt((1.0 - 1.0 / math.exp(1)) * (lnc(n,k) + alpha * alpha))
    lamba1 = (2*n*(1+1/3*eps1) * (lnc(n,k)+ell*math.log(n)+math.log(math.log(n)/math.log(2))))/(eps1*eps1)
    lamba2 = 2 * n * pow(((1.0 - 1.0 / math.exp(1)) * alpha + beta),2) / (eps*eps)
    LB=1
    # 初始化映射（b直接存储RR集，a和cover_counts实时更新）
    a = defaultdict(list)  # a[node] = [rrset_idx1, ...]
    b = []  # b[rr_idx] = [node1, ...]（替代原R）
    cover_counts = defaultdict(int)  # 节点覆盖计数
    
    len_b=0
    # 校准阶段
    for i in range(1, int(math.log(n-1) / math.log(2)) + 1):
        x = n / pow(2, i)
        ti = int(lamba1 / x)
        # 生成RR集并实时更新映射
        
        while len_b < ti:
            rrset = RRS_ori(G, node_list)
            b.append(rrset)
            for node in rrset:
                a[node].append(len_b)
                cover_counts[node] += 1
            len_b=len(b)
        
        # 无需复制，直接传递原始映射（函数内部不会修改它们）
        remaining_rrs = set(range(len(b)))
        SEED, js = optimized_nodeselect(k, a, b, cover_counts, remaining_rrs)
        
        if n * js / len(b) >= (1 + eps1) * x:
            LB = n * js / ((1 + eps1) * len(b))
            break
    
    # 最终采样阶段
    theta = int(lamba2 / LB)
    print(f"len(RR): {theta}")
    
    # 继续生成RR集至目标数量
    
    while len_b < theta:
        rrset = RRS_ori(G, node_list)
        b.append(rrset)
        for node in rrset:
            a[node].append(len_b)
            cover_counts[node] += 1
        len_b=len(b)
    
    # 最终种子选择（同样无需复制）
    remaining_rrs = set(range(len(b)))
    seed, js = optimized_nodeselect(k, a, b, cover_counts, remaining_rrs)
    
    print(f"IMM_RRS COST TIME: {time.time() - s:.2f}s")
    return seed

def optimized_nodeselect(k, a, b, cover_counts, remaining_rrs):
    """无需复制原始映射，仅在内部维护临时状态"""
    s = time.time()
    SEED = []
    total_covered = 0
    
    # 仅复制需要修改的状态（覆盖计数和剩余RR集）
    current_cover = cover_counts.copy()  # 复制需要修改的计数
    current_remaining = remaining_rrs.copy()  # 复制需要修改的集合
    
    for _ in range(k):
        if not current_remaining or not current_cover:
            break
        
        # 选择覆盖最多的节点
        seed = max(current_cover, key=lambda x: current_cover[x])
        SEED.append(seed)
        
        # 找到覆盖的RR集
        covered_rrs = [rr_idx for rr_idx in a[seed] if rr_idx in current_remaining]
        total_covered += len(covered_rrs)
        
        # 更新剩余RR集
        for rr_idx in covered_rrs:
            current_remaining.discard(rr_idx)
        
        # 更新其他节点的覆盖计数
        for rr_idx in covered_rrs:
            for node in b[rr_idx]:
                if node != seed and node in current_cover:
                    current_cover[node] -= 1
                    if current_cover[node] == 0:
                        del current_cover[node]
        
        # 移除已选节点
        del current_cover[seed]
    
    #print(f"节点选择耗时: {time.time() - s:.2f}秒")
    return SEED, total_covered
def forward_single(seed,g):#理论上不需要层级
    #s=time.time()
    result = []
    result.extend(seed)
    # 保存激活的状态
    checked_a = np.zeros(nodenum,dtype=bool)
    for t in result:
        checked_a[t] = 1
    # 当前节点不为空，进行影响
    while(len(result)>0):
        current_node = result.pop(0)
        for nbr in g.successors(current_node): # 得到当前节点的邻居节点
            if checked_a[nbr] == 0 :
                wt = g.get_edge_data(current_node,nbr)
                if random.uniform(0,1)<wt['weight'] :
                    result.append(nbr)
                    checked_a[nbr] = 1
    #print('forward:',time.time()-s)
    return (checked_a != 0).sum()
def pg_influence(G,k,seed_set):#转为列表存储，可以评估方差
    sss=0
    for i in range(k):
        sss+=forward_single(seed_set,G)
    print("influence:",sss/k)
    return sss/k
def calculate_two_hop_probability(graph, start_node):
    hop_probabilities = {}
    
    # 计算第一跳的激活概率
    first_hop_probs = {}
    for neighbor in graph.successors(start_node):  # 只考虑a的邻居
        # 计算第一跳传播概率
        hop_probabilities[neighbor] = graph[start_node][neighbor]['weight']  # 保存第一跳的概率
    # 计算第二跳的激活概率
    for node in graph.successors(start_node):
        for second_node in graph.successors(node):  # 第二跳从node的邻居开始
            # 计算第二跳的传播概率
            if(second_node not in hop_probabilities.keys()):hop_probabilities[second_node]=graph[node][second_node]['weight']*graph[start_node][node]['weight']
            else:
                hop_probabilities[second_node]=1-(1-hop_probabilities[second_node])*(1-graph[node][second_node]['weight']*graph[start_node][node]['weight']) 
    return hop_probabilities
def update_p_weight(F,flag,x0,temp):
    X=[]
    for i in F:
        X.append(temp[i])
    if flag == 1:
        return sum(X)-(len(X)-1)*x0
    elif x0==1:
        return 1
    else:
        m = len(X) - 1  # 这里假设 X 里有 m+1 个元素，X[0] 是 X_0，X[m] 是 X_m
        sum_part1 = sum(1 - X[j] for j in range(m))  # 计算第一部分的和
        sum_part2 = 0
        for i in range(m):
            for j in range(i+1, m):
                sum_part2 += (1 - x0) + (1 - X[i]) * (1 - X[j]) / (1 - x0)  # 计算第二部分的和
        part3 = (m - 1) * math.sqrt(1 - x0)  # 第三部分
        result = 1 - (math.sqrt(sum_part1 + sum_part2) - part3) ** 2  # 最终计算结果
        return max(result,max(X))
def reverse_dict(Ir):
    I = {}
    # 遍历 Ir 字典
    for key1, value1 in Ir.items():
        for key2, value2 in value1.items():
            if key2 not in I:
                I[key2] = {}
            I[key2][key1] = value2
    return I
def insert_list(new_tuple,new_sorted_list,k):
    for i in range(0,k-1):
        if(new_sorted_list[k-2-i][1]>new_tuple[1]):
            new_sorted_list.insert(k-1-i,new_tuple)
            new_sorted_list=new_sorted_list[:k]
            return new_sorted_list
    new_sorted_list.insert(0,new_tuple)
    new_sorted_list=new_sorted_list[:k]
    return new_sorted_list

def update_index_table(If,Ir,F,sh):
    t=time.time()
    new_forward_label={}
    for node in sh:
        new_forward_label[node]={}
        for key,value in If[node].items():
            flag=value[0]
            x0=value[1]
            temp_dict=value[2]
            new_forward_label[node][key]=update_p_weight(F,flag,x0,temp_dict)
    
    print(f"new_forward_label cost time:{time.time()-t}")
    return new_forward_label

def update_node_influence(G,node,new_forward_label):
    temp={}
    if node in new_forward_label.keys():
        for key,value in new_forward_label[node].items():
            if key in temp.keys(): temp[key]=max(value,temp[key])
            else:temp[key]=value
            if key in new_forward_label.keys():
                for key1,value1 in new_forward_label[key].items():
                    if key1 in temp.keys(): temp[key1]=max(value1*temp[key],temp[key1])
                    else:temp[key]=value1*temp[key]
    else:
        t=0
        temp=calculate_two_hop_probability(G, node)#获取两跳内的影响力 calculate_two_hop_probability
        temp1=temp.copy()
        if len(temp)!=0:
            for key,value in temp1.items():
                if key in new_forward_label.keys():
                    for key1,value1 in new_forward_label[key].items():
                        if key1 in temp.keys(): temp[key1]=max(value1*value,temp[key1])
                        else:
                            temp[key1]=value1*value
    return sum(temp.values())

def prune_labels(new_forward_label):
    t=time.time()
    best_for_b = {}
    pruned = {}
    for a, b_dict in new_forward_label.items():
        pruned[a]={}
        for b, value in b_dict.items():
            # 如果 b 尚未出现，或当前 value 更大，则更新
            if b not in best_for_b or value > best_for_b[b][1]:
                best_for_b[b] = (a, value)
    for b, (a, value) in best_for_b.items():
        pruned[a][b] = value
    for key,temp in pruned.items():
        pruned[key]=sum(temp.values())+1
    print(f"jz cost time:{time.time()-t}")
    return pruned

def update_node_influence2(G,node,pruned,new_forward_label):
    temp=0
    if node in new_forward_label.keys():
        for key in new_forward_label[node].keys():
            if key in pruned.keys(): temp+=new_forward_label[node][key]*pruned[key]
            else: temp+=new_forward_label[node][key]
    else:
        for nbr in G.successors(node): # 得到当前节点的邻居节点
            value = G.get_edge_data(node,nbr)['weight']
            if nbr in pruned.keys():
                temp+=value*pruned[nbr]
            else:
                temp+=value
    return temp

def test(F,G,If,Ir,sh,sorted_list,k):
    repeat=100
    t1=time.time()
    newG=update_weight(G,F)
    seed4=IMM_RRS(newG,k,0.5,1)
    print('time:',time.time()-t1,'IMM influence:')
    pg_influence(newG,repeat,seed4)

    t2=time.time()
    new_forward_label=update_index_table(If,Ir,F,sh)
    tt1=time.time()-t2
    tt2=time.time()-t2-tt1
    new_sorted_list=[]
    for i in range(k):
        node=sorted_list[i][0]
        yxl=update_node_influence(newG,node,new_forward_label)
        new_sorted_list.append((node,yxl))
    new_sorted_list = sorted(new_sorted_list, key=lambda x: x[1], reverse=True)
    temp=0
    for i in range(k,2*k):
        node=sorted_list[i][0]
        yxl=update_node_influence(newG,node,new_forward_label)
        if(yxl>new_sorted_list[k-1][1]):
            temp=0
            new_tuple=(node,yxl)
            new_sorted_list=insert_list(new_tuple,new_sorted_list,k)
        else:
            temp=temp+1
            if(temp>=k/2):
                print(i)
                break
    seed3=[item[0] for item in new_sorted_list[:k]]
    print('time:',time.time()-t2-tt2,'my method1 influence:')
    pg_influence(newG,repeat,seed3)
    
def qureies(newG,sorted_list,k,new_forward_label,pruned,tt1,tt2):
    repeat=100
    t1=time.time()
    seed4=IMM_RRS(newG,k,0.5,1)
    imm_t=time.time()-t1
    imm_i=pg_influence(newG,repeat,seed4)
    t2=time.time()
    new_sorted_list=[]
    for i in range(k):
        node=sorted_list[i][0]
        yxl=update_node_influence(newG,node,new_forward_label)
        new_sorted_list.append((node,yxl))
    new_sorted_list = sorted(new_sorted_list, key=lambda x: x[1], reverse=True)
    temp=0
    for i in range(k,5*k):
        node=sorted_list[i][0]
        
        yxl=update_node_influence(newG,node,new_forward_label)
        if(yxl>new_sorted_list[k-1][1]):
            temp=0
            new_tuple=(node,yxl)
            new_sorted_list=insert_list(new_tuple,new_sorted_list,k)
        else:
            temp=temp+1
            if(temp>=k):
                print(i)
                break
    seed3=[item[0] for item in new_sorted_list[:k]]
    m1_t=time.time()-t2+tt1
    m1_i=pg_influence(newG,repeat,seed3)
    
    
    t3=time.time()
    new_sorted_list=[]
    for i in range(k):
        node=sorted_list[i][0]
        yxl=update_node_influence2(newG,node,pruned,new_forward_label)
        new_sorted_list.append((node,yxl))
    new_sorted_list = sorted(new_sorted_list, key=lambda x: x[1], reverse=True)

    temp=0
    for i in range(k,10*k):
        node=sorted_list[i][0]
        yxl=update_node_influence2(newG,node,pruned,new_forward_label)
        if(yxl>new_sorted_list[k-1][1]):
            temp=0
            new_tuple=(node,yxl)
            new_sorted_list=insert_list(new_tuple,new_sorted_list,k)
        else:
            temp=temp+1
            if(temp>=2*k):
                break
    seed3=[item[0] for item in new_sorted_list[:k]]
    m2_t=time.time()-t3+tt1+tt2
    m2_i=pg_influence(newG,repeat,seed3)
    return imm_i,round(imm_t, 3),m1_i,round(m1_t, 3),m2_i,round(m2_t, 3)

def test_quries(G,If,Ir,sh,sorted_list,k,len_topic,random_num=10):
    imm_i_list=[]
    imm_t_list=[]
    m1_i_list=[]
    m1_t_list=[]
    m2_i_list=[]
    m2_t_list=[]
    for i in range(random_num):
        print(i,'#########################################################################')
        F=random.sample(community_list, len_topic)

        newG=update_weight(G,F)
        t2=time.time()
        new_forward_label=update_index_table(If,Ir,F,sh)
        tt1=time.time()-t2
        pruned=prune_labels(new_forward_label)
        tt2=time.time()-t2-tt1
        #将方向取正
        print('new_forward_label',tt1)
        print('prune_labels',tt2)

        imm_i,imm_t,m1_i,m1_t,m2_i,m2_t=qureies(newG,sorted_list,k,new_forward_label,pruned,tt1,tt2)
        imm_i_list.append(imm_i)
        imm_t_list.append(imm_t)
        m1_i_list.append(m1_i)
        m1_t_list.append(m1_t)
        m2_i_list.append(m2_i)
        m2_t_list.append(m2_t)
    return imm_i_list,imm_t_list,m1_i_list,m1_t_list,m2_i_list,m2_t_list

imm_i_list,imm_t_list,m1_i_list,m1_t_list,m2_i_list,m2_t_list=test_quries(G,If,Ir,sh,sorted_list,k=50,len_topic=2,random_num=10)
print('imm_i_list=',imm_i_list)
print('imm_t_list=',imm_t_list)
print('m1_i_list=',m1_i_list)
print('m1_t_list=',m1_t_list)
print('m2_i_list=',m2_i_list)
print('m2_t_list=',m2_t_list)

In [1]:
import math
def calculate_mean_and_variance(data_list):
    """
    计算列表的均值和方差
    
    参数:
    data_list: 包含数值的列表
    
    返回:
    一个元组 (均值, 方差)
    """
    # 计算均值
    n = len(data_list)
    if n == 0:
        return (0, 0)  # 空列表处理
    
    mean = sum(data_list) / n
    
    # 计算方差
    variance = sum((x - mean) **2 for x in data_list) / n
    
    return (mean, variance)

In [6]:
#wiki_2
imm_i_list=[710.62, 16050.1, 955.93, 216.31, 1255.28, 198.72, 15960.13, 231.31, 16354.01, 202.28]
imm_t_list=[527.8745138645172, 8768.433153867722, 258.64855337142944, 970.3099462985992, 1085.7751395702362, 1634.8815093040466, 8995.46716761589,
895.7688150405884, 8873.08875799179, 1530.8392066955566]
m1_i_list=[284.79, 15135.92, 495.39, 135.45, 898.53, 134.29, 15345.3, 137.59, 15735.9, 136.72]
m1_t_list=[72.11591863632202, 72.30710244178772, 72.25917673110962, 37.19414258003235, 70.07927203178406, 44.041483879089355, 69.30497550964355, 44.35485100746155, 68.97237849235535, 40.26675772666931]
m2_i_list=[431.7, 15386.25, 579.44, 160.45, 934.71, 162.42, 15461.71, 160.53, 15782.55, 161.57]
m2_t_list=[35.36506628990173, 33.5197811126709, 32.55778932571411, 27.60662531852722, 30.076689958572388, 30.09136462211609, 30.392477989196777, 30.85953450202942, 29.819267749786377, 30.640400648117065]

In [7]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 5213.4690
  方差: 51115772.4564
  标准差: 7149.5295
imm_t_list:
  均值: 3354.1087
  方差: 13231977.1135
  标准差: 3637.5785
m1_i_list:
  均值: 4843.9880
  方差: 47875853.4335
  标准差: 6919.2379
m1_t_list:
  均值: 59.0896
  方差: 211.7890
  标准差: 14.5530
m2_i_list:
  均值: 4922.1330
  方差: 48411591.4585
  标准差: 6957.8439
m2_t_list:
  均值: 31.0929
  方差: 4.2849
  标准差: 2.0700


In [8]:
#twitter_2
imm_i_list=[3526.39, 15832.09, 4125.12, 3618.61, 4137.69, 3582.29, 3617.47, 3600.84, 3657.75, 3703.37]
imm_t_list=[1823.7197427749634, 5542.651782989502, 1062.6834468841553, 1777.7096002101898, 1162.3165936470032, 1742.68949842453, 1740.1295001506805, 1734.4889504909515, 1161.7768142223358, 1671.8071699142456]
m1_i_list=[3161.24, 15155.56, 3806.44, 3234.48, 3772.78, 3211.56, 3233.6, 3181.95, 3162.8, 3243.86]
m1_t_list=[1165.1246767044067, 2772.652949333191, 1556.9714722633362, 1364.1005997657776, 1818.5133483409882, 1226.9722628593445, 1187.065824508667, 1203.439442873001, 1191.0057697296143, 1188.067779302597]
m2_i_list=[3087.74, 15321.12, 3725.22, 3110.29, 3719.54, 3124.26, 3141.21, 3128.92, 3106.35, 3126.16]
m2_t_list=[113.46837544441223, 115.49876093864441, 114.59983611106873, 115.14187335968018, 134.14063262939453, 149.30387091636658, 123.4554693698883, 127.42534160614014, 118.74908924102783, 122.66498517990112]

In [9]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 4940.1620
  方差: 13224895.0190
  标准差: 3636.6049
imm_t_list:
  均值: 1941.9973
  方差: 1519206.8227
  标准差: 1232.5611
m1_i_list:
  均值: 4516.4270
  方差: 12630904.7996
  标准差: 3553.9984
m1_t_list:
  均值: 1467.3914
  方差: 229751.8213
  标准差: 479.3243
m2_i_list:
  均值: 4459.0810
  方差: 13166359.6307
  标准差: 3628.5479
m2_t_list:
  均值: 123.4448
  方差: 113.1436
  标准差: 10.6369


In [2]:
#soc_2
imm_i_list=[1050.81, 812.06, 717.98, 1067.77, 4828.63, 1568.67, 1044.57, 711.8, 749.48, 713.12, 789.55, 825.78, 3957.97, 858.15, 736.7, 740.76, 1064.51, 1176.56, 713.31, 875.48]
imm_t_list=[1001.3591666221619, 183.27918004989624, 102.34314393997192, 1109.3682627677917, 1921.6621325016022, 1775.2965421676636, 1053.4418632984161, 106.80237007141113, 185.68676471710205, 98.74900531768799, 162.80653023719788, 196.98271417617798, 1274.0460379123688, 893.792275428772, 100.18861865997314, 105.25448155403137, 1060.135704755783, 1241.15100979805, 93.35813403129578, 821.4598319530487]
m1_i_list=[834.61, 691.5, 635.43, 827.55, 4226.04, 1304.72, 795.93, 619.81, 642.55, 635.64, 706.36, 689.76, 3083.08, 681.11, 632.83, 647.78, 825.01, 988.35, 634.92, 698.0]
m1_t_list=[433.81705141067505, 415.27505445480347, 290.1373631954193, 443.3808286190033, 446.7507219314575, 440.9163029193878, 443.3714623451233,
272.44285249710083, 344.92409801483154, 267.7449221611023, 429.3897602558136, 425.61887407302856, 448.32267451286316, 447.31700587272644, 261.4091191291809, 302.58216381073, 463.3520972728729, 454.1658978462219, 276.9022355079651, 470.73250341415405]
m2_i_list=[1016.7, 801.87, 710.46, 1037.86, 4654.99, 1505.86, 1021.87, 708.86, 755.83, 712.2, 775.35, 815.78, 3756.6, 850.4, 721.19, 739.31, 1038.38, 1141.24, 703.19, 851.23]
m2_t_list=[23.4939923286438, 24.825259923934937, 31.02479076385498, 30.369807481765747, 30.6595516204834, 31.86467218399048, 29.973726987838745, 31.631683588027954, 29.70013451576233, 28.98369789123535, 31.10701894760132, 29.18765425682068, 30.613106966018677, 31.43734836578369, 30.542978286743164, 33.56948256492615, 33.67642664909363, 32.95697641372681, 30.709240198135376, 31.20528793334961]

In [3]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 1250.1830
  方差: 1159800.6620
  标准差: 1076.9404
imm_t_list:
  均值: 674.3582
  方差: 352424.1340
  标准差: 593.6532
m1_i_list:
  均值: 1040.0490
  方差: 816875.3903
  标准差: 903.8116
m1_t_list:
  均值: 388.9276
  方差: 5865.4216
  标准差: 76.5860
m2_i_list:
  均值: 1215.9585
  方差: 1050877.3831
  标准差: 1025.1231
m2_t_list:
  均值: 30.3766
  方差: 5.8366
  标准差: 2.4159


In [12]:
#gnu_2
imm_i_list=[62.1, 63.09, 62.23, 61.0, 65.82, 65.05, 67.14, 64.26, 59.2, 66.46, 67.49, 61.79, 67.26, 61.33, 63.0, 60.97, 66.27, 65.96, 61.03, 62.36,64.48, 66.8, 76.76, 59.96, 64.23, 63.23, 66.66, 66.16, 64.91, 64.0, 63.65, 67.28, 63.37, 62.55, 63.03, 63.64, 61.36, 61.61, 64.29, 63.88, 60.18, 77.36, 67.74, 63.08, 63.56, 64.21, 63.34, 64.32, 61.34, 60.37]
imm_t_list=[5.266377687454224, 5.061373233795166, 5.244128227233887, 5.58790135383606, 5.40909743309021, 5.350801467895508, 5.155135631561279, 5.413572072982788, 5.733850479125977, 5.142795562744141, 5.480563402175903, 5.583791971206665, 5.24826192855835, 5.508958578109741, 5.673229932785034, 5.614293336868286, 5.4840028285980225, 5.319877624511719, 5.608705520629883, 5.644229888916016, 5.32597279548645, 5.465412378311157, 5.152255535125732, 5.489980936050415, 5.449573993682861, 5.57381796836853, 5.4544243812561035, 5.421292066574097, 5.4113545417785645, 5.616291046142578, 5.410325527191162, 5.271710634231567, 5.7098236083984375, 5.54596209526062, 5.2784154415130615, 5.6182334423065186, 5.620317697525024, 5.593677282333374, 5.355917453765869, 5.45186972618103, 5.431272745132446, 4.992943525314331, 5.238469362258911, 5.656707525253296, 5.524636268615723, 5.3407673835754395, 5.49726128578186, 5.351980686187744, 5.345754384994507, 5.369276285171509]
m1_i_list=[63.77, 64.4, 63.48, 64.21, 64.04, 66.27, 65.98, 65.07, 64.96, 66.72, 63.85, 64.87, 65.37, 63.86, 63.16, 62.11, 68.06, 65.82, 65.28, 64.04, 64.7, 64.75, 69.46, 63.36, 64.62, 63.98, 64.03, 64.83, 63.25, 64.87, 64.0, 63.98, 63.52, 63.06, 64.48, 63.08, 64.49, 62.5, 64.57, 64.99, 64.46, 68.63, 63.65, 63.55, 65.35, 62.72, 63.43, 63.4, 63.89, 64.12]
m1_t_list=[0.3959999084472656, 0.37354421615600586, 0.44898533821105957, 0.36245203018188477, 0.4028041362762451, 0.33883094787597656, 0.3732428550720215, 0.3627355098724365, 0.36679506301879883, 0.34345436096191406, 0.3561539649963379, 0.33704185485839844, 0.35955333709716797, 0.44063901901245117, 0.3524312973022461, 0.3564131259918213, 0.33395886421203613, 0.3669130802154541, 0.3417243957519531, 0.3310580253601074, 0.36188316345214844, 0.3329646587371826, 0.3742332458496094, 0.3467838764190674, 0.3336977958679199, 0.36971306800842285, 0.34709620475769043, 0.3587782382965088, 0.3418009281158447, 0.3640859127044678, 0.37091970443725586, 0.36634159088134766, 0.37359142303466797, 0.35069799423217773, 0.36899805068969727, 0.32561349868774414, 0.31322526931762695, 0.31693077087402344, 0.35347819328308105, 0.36751627922058105, 0.35065793991088867, 0.3586430549621582, 0.32334065437316895, 0.34851717948913574, 0.33307313919067383, 0.3040297031402588, 0.32588696479797363, 0.3524203300476074, 0.3195180892944336, 0.36134815216064453]
m2_i_list=[63.89, 63.22, 63.04, 63.36, 62.77, 65.72, 64.1, 63.99, 62.7, 65.45, 63.67, 63.91, 64.75, 62.96, 61.82, 61.83, 68.03, 67.68, 65.83, 62.63, 63.33, 63.84, 68.92, 64.09, 64.1, 62.5, 63.11, 64.75, 62.23, 63.79, 63.84, 62.89, 63.76, 63.84, 62.25, 62.94, 64.69, 61.52, 64.28, 63.14, 64.4, 68.27, 63.56, 63.72, 63.83, 64.83, 62.24, 62.26, 64.99, 62.84]
m2_t_list=[0.32897090911865234, 0.32294535636901855, 0.4021744728088379, 0.30817437171936035, 0.35030341148376465, 0.2989485263824463, 0.3161637783050537, 0.30665016174316406, 0.31343936920166016, 0.2895784378051758, 0.27646613121032715, 0.2803323268890381, 0.2970871925354004, 0.38924217224121094, 0.28838014602661133, 0.2987804412841797, 0.27823686599731445, 0.28388500213623047, 0.2656676769256592, 0.27722620964050293, 0.30649423599243164, 0.2769176959991455, 0.3122670650482178, 0.29558229446411133, 0.2567472457885742, 0.2842118740081787, 0.28817248344421387, 0.3016321659088135, 0.2805495262145996, 0.3140294551849365, 0.30993032455444336, 0.30149364471435547, 0.29669189453125, 0.265073585510254, 0.3140254020690918, 0.2676663398742676, 0.25528979301452637, 0.25317931175231934, 0.29950594902038574, 0.30811715126037597, 0.29633021354675293, 0.30185532569885254, 0.27411580085754395, 0.2844552993774414, 0.2801072597503662, 0.27810144424438477, 0.26959019389038086, 0.295443058013916, 0.264803409576416, 0.3015899658203125]

In [13]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 64.2232
  方差: 11.5924
  标准差: 3.4048
imm_t_list:
  均值: 5.4299
  方差: 0.0289
  标准差: 0.1699
m1_i_list:
  均值: 64.5008
  方差: 2.0143
  标准差: 1.4193
m1_t_list:
  均值: 0.3552
  方差: 0.0007
  标准差: 0.0271
m2_i_list:
  均值: 63.9220
  方差: 2.5688
  标准差: 1.6027
m2_t_list:
  均值: 0.2961
  方差: 0.0008
  标准差: 0.0283


In [14]:
#flixster_2
imm_i_list=[273.66, 356.88, 227.08, 251.53, 269.17, 263.62, 263.49, 386.47, 285.9, 246.4, 269.56, 243.07, 232.4, 236.47, 392.63, 225.78, 360.62, 247.82, 262.68, 242.15, 385.99, 240.97, 224.47, 234.37, 226.26, 276.28, 228.03, 263.45, 256.66, 256.62, 245.03, 231.84, 365.91, 220.35, 225.36, 257.27, 234.37, 237.05, 223.97, 266.42, 247.49, 223.91, 10940.35, 360.95, 226.36, 250.18, 10738.09, 11023.3, 222.98, 242.18]
imm_t_list=[48.88920450210571, 27.36911702156067, 48.15269899368286, 45.79902219772339, 46.319605112075806, 43.40947604179382, 46.12977361679077, 24.108712673187256, 46.36579990386963, 43.058611154556274, 44.158737659454346, 42.62364435195923, 42.437674045562744, 43.06496214866638,
24.406965494155884, 42.626540660858154, 26.697829484939575, 43.224241495132446, 44.268959522247314, 42.50404500961304, 26.140802145004272, 42.56340575218201, 44.76560616493225, 43.53553915023804, 42.492164611816406, 43.953872203826904, 42.6384220123291, 43.4731080532074,
43.16734504699707, 43.090062618255615, 43.03535223007202, 44.328707456588745, 24.666492462158203, 42.943060874938965, 42.73170232772827, 42.77950310707092, 45.10602784156799, 43.412007093429565, 43.229430198669434, 44.792792081832886, 42.88694429397583, 44.56794548034668, 9.967475414276123, 24.341068983078003, 42.93878245353699, 43.36198902130127, 9.184160470962524, 9.654539823532104, 45.089850187301636,
42.466930627822876]
m1_i_list=[206.56, 262.91, 197.08, 218.46, 214.44, 222.27, 221.25, 304.33, 214.48, 209.37, 223.51, 197.02, 199.66, 208.53, 302.93, 195.14, 279.99, 201.57, 211.02, 202.93, 312.55, 202.89, 196.98, 197.67, 195.17, 226.3, 203.46, 215.81, 216.23, 221.13, 210.19, 203.32, 282.19, 190.79,
201.23, 210.74, 201.03, 199.19, 194.47, 219.34, 202.82, 195.33, 9028.45, 274.18, 195.25, 216.32, 8913.37, 9056.14, 195.15, 196.22]
m1_t_list=[15.765514612197876, 16.12040901184082, 16.07689619064331, 15.929808139801025, 15.819876432418823, 15.938900709152222, 15.775610208511353, 15.837315797805786, 16.04150414466858, 15.88817310333252, 15.941092491149902, 16.0432448387146, 16.295999765396118, 15.66063117980957, 16.201358556747437, 15.75469160079956, 15.802785158157349, 15.896167516708374, 16.046466588974, 16.057108879089355, 16.07737684249878, 16.120004653930664, 15.73298954963684, 15.798336029052734, 15.947649955749512, 15.883325815200806, 16.033259630203247, 16.279308319091797, 15.89729118347168, 16.180442571640015, 16.320127248764038, 15.954345703125, 16.213791370391846, 15.59045124053955, 15.802826881408691, 16.24907684326172, 15.948265075683594, 15.898551940917969, 16.088817596435547, 15.956042766571045, 16.120592832565308, 15.99221658706665, 16.321382522583008, 16.406999588012695, 15.606171369552612, 16.056334257125854, 16.221063137054443, 16.576337575912476, 16.331579208374023, 16.082709074020386]
m2_i_list=[260.19, 288.31, 224.46, 254.16, 262.67, 262.9, 264.34, 372.11, 270.36, 244.57, 267.59, 222.36, 225.17, 239.46, 366.01, 221.11, 298.61,
229.22, 265.83, 225.7, 375.44, 226.29, 220.51, 222.87, 225.46, 264.09, 233.44, 245.38, 242.61, 250.11, 249.35, 228.04, 308.82, 217.96, 222.41, 238.66, 233.9, 225.03, 224.07, 266.09, 234.28, 223.38, 8902.85, 287.9, 222.51, 250.03, 8870.68, 9019.45, 226.47, 220.54]
m2_t_list=[6.011584997177124, 7.029104709625244, 6.661187648773193, 6.564594984054565, 6.401021957397461, 6.342846155166626, 6.408684968948364, 6.191713333129883, 6.361341714859009, 6.182476043701172, 6.1365580558776855, 6.239428997039795, 6.202719211578369, 5.99385929107666, 6.380551338195801, 5.985346555709839, 6.157739877700806, 6.1205902099609375, 6.24814772605896, 6.417182922363281, 6.373514890670776, 6.263838529586792, 6.354033470153809, 6.306335687637329, 6.245511054992676, 6.330016374588013, 6.375545024871826, 6.355888366699219, 6.141449213027954, 6.240544080734253, 6.281735181808472, 6.4216766357421875, 6.403414011001587, 5.877395868301392, 6.225692272186279, 6.464435338973999, 6.379651308059692, 6.32551121711731, 6.304964780807495, 6.2307891845703125, 6.319548845291138, 6.243568658828735, 6.25074577331543, 6.76520299911499, 5.969684600830078, 6.2864134311676025, 6.370405197143555, 6.819289922714233, 6.7860448360443115, 6.261301279067993]

In [15]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 902.2768
  方差: 6383887.1474
  标准差: 2526.6355
imm_t_list:
  均值: 39.2584
  方差: 99.1037
  标准差: 9.9551
m1_i_list:
  均值: 745.3472
  方差: 4349751.3942
  标准差: 2085.6058
m1_t_list:
  均值: 16.0116
  方差: 0.0443
  标准差: 0.2106
m2_i_list:
  均值: 772.8750
  方差: 4249808.6259
  标准差: 2061.5064
m2_t_list:
  均值: 6.3202
  方差: 0.0460
  标准差: 0.2145


# len_topic=4

In [4]:
#soc_4
imm_i_list= [1560.31, 899.9, 1041.49, 1062.95, 1143.9, 771.04, 1216.51, 4018.06, 1255.11, 815.1, 1256.83, 4121.57, 743.22, 1046.22, 796.39, 912.62, 1191.08, 772.73, 928.8, 1631.53]
imm_t_list= [1993.002, 300.362, 790.881, 801.434, 1007.592, 111.493, 1325.707, 1329.902, 1394.765, 179.784, 1283.835, 1381.775, 102.283, 592.305, 141.117, 901.944, 1326.957, 107.064, 253.695, 2411.837]
m1_i_list= [1372.93, 759.66, 835.31, 864.66, 908.26, 656.17, 1000.96, 3122.16, 1034.96, 736.4, 1023.05, 3277.76, 654.42, 862.46, 698.28, 729.81, 945.18, 670.9, 770.28, 1419.55]
m1_t_list= [503.827, 602.343, 655.701, 641.544, 640.286, 447.476, 646.862, 661.341, 628.151, 358.495, 643.22, 643.667, 479.259, 658.508, 418.301, 630.403, 633.788, 433.411, 590.018, 653.436]
m2_i_list= [1533.0, 885.54, 1018.57, 1021.04, 1100.04, 741.22, 1181.25, 3804.34, 1205.78, 801.25, 1225.78, 3901.16, 727.97, 1014.76, 782.86, 878.32, 1162.4, 739.37, 896.65, 1574.19]
m2_t_list= [51.366, 34.443, 51.066, 47.225, 51.141, 49.423, 46.329, 53.174, 43.958, 49.533, 48.2, 51.99, 52.856, 51.032, 49.475, 48.757, 52.979, 52.554, 52.621, 51.394]

In [5]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 1359.2680
  方差: 874776.5768
  标准差: 935.2949
imm_t_list:
  均值: 886.8867
  方差: 426802.0230
  标准差: 653.3009
m1_i_list:
  均值: 1117.1580
  方差: 524993.7072
  标准差: 724.5645
m1_t_list:
  均值: 578.5018
  方差: 9118.3032
  标准差: 95.4898
m2_i_list:
  均值: 1309.7745
  方差: 773666.3165
  标准差: 879.5830
m2_t_list:
  均值: 49.4758
  方差: 17.5795
  标准差: 4.1928


In [6]:
#twittter_4
imm_i_list= [4216.65, 15789.6, 3865.09, 3747.74, 3757.36, 3534.97, 3786.41, 4038.24, 4128.44, 16554.61]
imm_t_list= [1024.412, 5611.72, 1059.471, 1658.624, 1094.056, 1659.92, 1682.326, 1012.877, 998.235, 5533.468]
m1_i_list= [3769.6, 15155.19, 3211.3, 3236.29, 3181.34, 3153.43, 3301.68, 3623.05, 3737.29, 15473.64]
m1_t_list= [1603.383, 3047.735, 1267.003, 1262.255, 1300.195, 1287.449, 1316.101, 1287.482, 1284.61,3018.349]
m2_i_list= [3697.16, 15281.64, 3121.28, 3118.64, 3146.12, 3087.8, 3125.35, 3486.16, 3553.68, 15797.47]
m2_t_list= [138.818, 148.466, 149.411, 148.221, 150.638, 150.002, 149.767, 149.406, 149.346, 145.149]

In [7]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 6341.9110
  方差: 24223490.9021
  标准差: 4921.7366
imm_t_list:
  均值: 2133.5109
  方差: 3031982.0820
  标准差: 1741.2588
m1_i_list:
  均值: 5784.2810
  方差: 22759012.0360
  标准差: 4770.6406
m1_t_list:
  均值: 1667.4562
  方差: 475242.5419
  标准差: 689.3784
m2_i_list:
  均值: 5741.5300
  方差: 24055655.4040
  标准差: 4904.6565
m2_t_list:
  均值: 147.9224
  方差: 11.2512
  标准差: 3.3543


In [8]:
#wiki_4
imm_i_list= [16217.95, 1144.82, 1621.19, 1259.77, 16160.47, 1011.81, 2021.59, 16672.31, 1028.63, 1711.48]
imm_t_list= [9011.508, 283.361, 302.067, 313.263, 9195.961, 280.751, 1330.519, 9185.22, 271.289, 189.613]
m1_i_list= [15496.28, 598.38, 911.74, 610.27, 15510.89, 695.53, 1700.21, 15984.0, 549.04, 1084.89]
m1_t_list= [74.595, 77.502, 79.352, 80.407, 77.68, 50.707, 77.334, 76.135, 76.034, 76.117]
m2_i_list= [15588.66, 722.89, 1024.11, 710.89, 15573.55, 766.89, 1692.37, 16027.75, 624.93, 1268.79]
m2_t_list= [37.076, 36.748, 35.221, 39.465, 35.495, 36.65, 36.744, 35.793, 36.255, 34.915]

In [9]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 5885.0020
  方差: 47043984.7532
  标准差: 6858.8618
imm_t_list:
  均值: 3036.3552
  方差: 16017503.6971
  标准差: 4002.1874
m1_i_list:
  均值: 5314.1230
  方差: 46022629.1023
  标准差: 6783.9980
m1_t_list:
  均值: 74.5863
  方差: 65.9273
  标准差: 8.1196
m2_i_list:
  均值: 5400.0830
  方差: 45835092.4086
  标准差: 6770.1619
m2_t_list:
  均值: 36.4362
  方差: 1.4944
  标准差: 1.2224


In [10]:
#gnu_4
imm_i_list= [65.86, 67.52, 71.5, 69.64, 70.08, 64.22, 71.13, 73.81, 71.23, 69.22, 68.43, 70.71, 74.08, 65.26, 74.02, 68.37, 69.19, 70.75, 70.69, 64.53, 70.56, 67.1, 69.42, 63.35, 70.3, 64.65, 63.57, 66.75, 66.2, 67.38, 63.31, 65.07, 67.07, 67.55, 66.01, 73.45, 68.1, 72.92, 66.57, 64.79]
imm_t_list= [5.509, 5.297, 5.509, 5.334, 5.189, 5.455, 5.606, 5.352, 5.858, 5.652, 5.57, 5.779, 5.486, 5.73, 5.703, 5.586, 5.978, 5.475, 5.936, 6.251, 5.917, 5.584, 5.802, 6.025, 5.46, 5.625, 5.648, 5.94, 5.375, 5.604, 5.598, 5.633, 5.873, 5.744, 5.517, 5.301, 5.527, 5.463, 5.388, 5.663]
m1_i_list= [65.17, 65.48, 69.94, 68.02, 65.23, 65.79, 66.82, 68.13, 64.98, 67.73, 65.94, 65.2, 68.75, 65.1, 68.14, 65.42, 66.63, 69.57, 68.62, 65.26, 68.8, 65.86, 66.62, 65.29, 67.04, 64.29, 65.68, 64.45, 69.56, 65.53, 64.79, 65.4, 64.66, 66.35, 66.1, 67.75, 64.98, 68.07, 66.1, 65.8]
m1_t_list= [0.455, 0.432, 0.498, 0.387, 0.471, 0.486, 0.384, 0.419, 0.455, 0.41, 0.371, 0.379, 0.431, 0.417, 0.417, 0.379, 0.453, 0.475, 0.614, 0.613, 0.662, 0.62, 0.566, 0.555, 0.422, 0.494, 0.419, 0.464, 0.428, 0.39, 0.408, 0.417, 0.41, 0.391, 0.367, 0.386, 0.431, 0.413, 0.412, 0.39]
m2_i_list= [62.97, 64.59, 70.7, 67.46, 63.99, 65.25, 65.12, 66.65, 64.5, 67.25, 64.87, 63.04, 67.16,64.58, 66.95, 64.0, 65.19, 69.25, 67.34, 64.39, 68.2, 64.33, 66.14, 63.5, 68.15, 63.06, 63.59, 62.76, 69.87, 64.42, 64.2, 64.1, 63.97, 65.47, 66.31, 65.9, 64.98, 68.34, 64.91, 64.86]
m2_t_list= [0.39, 0.379, 0.448, 0.339, 0.412, 0.415, 0.326, 0.365, 0.387, 0.358, 0.32, 0.326, 0.371,0.364, 0.363, 0.307, 0.401, 0.349, 0.54, 0.487, 0.613, 0.563, 0.508, 0.502, 0.358, 0.419, 0.361, 0.403, 0.357, 0.334, 0.355, 0.363, 0.355, 0.331, 0.318, 0.334, 0.38, 0.358, 0.35, 0.327]

In [11]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 68.3590
  方差: 9.4881
  标准差: 3.0803
imm_t_list:
  均值: 5.6235
  方差: 0.0506
  标准差: 0.2249
m1_i_list:
  均值: 66.4760
  方差: 2.4052
  标准差: 1.5509
m1_t_list:
  均值: 0.4498
  方差: 0.0055
  标准差: 0.0742
m2_i_list:
  均值: 65.5577
  方差: 3.8098
  标准差: 1.9519
m2_t_list:
  均值: 0.3884
  方差: 0.0050
  标准差: 0.0706


In [12]:
#flixster_4
imm_i_list= [286.35, 442.61, 10909.31, 254.44, 253.65, 295.04, 259.09, 442.61, 11042.14, 428.68, 382.84, 279.57, 313.76, 277.75, 11139.93, 388.4, 256.39, 566.43, 552.81, 299.99, 381.36, 445.63, 582.27,11061.46, 471.49, 445.37, 414.02, 310.73, 282.35, 11050.88, 278.24, 10902.04, 257.16, 441.63, 400.55, 272.0, 284.18, 442.11, 386.45, 433.22]
imm_t_list= [49.686, 24.364, 9.377, 42.956, 43.187, 42.642, 44.247, 22.831, 8.324, 24.673, 24.578, 41.465, 27.889, 41.437, 8.733, 24.282, 43.115, 24.135, 26.592, 41.298, 24.276, 25.086, 23.781, 10.893,22.876, 24.76, 24.593, 28.62, 41.866, 8.934, 44.404, 8.602, 42.038, 25.431, 21.565, 44.303, 41.686, 22.293, 24.267, 24.489]
m1_i_list= [225.18, 313.2, 8896.8, 207.11, 207.13, 221.28, 213.51, 316.52, 8975.76, 304.2, 284.94, 226.39, 239.58, 226.73, 9026.82, 281.25, 200.53, 402.53, 402.46, 230.37, 286.33, 346.03, 422.68, 9021.23, 325.18, 344.01, 312.4, 230.8, 222.64, 9209.86, 228.11, 8845.03, 218.13, 349.51, 282.89, 221.64, 237.06, 295.99, 285.51, 308.71]
m1_t_list= [18.11, 19.054, 19.703, 18.818, 18.869, 17.65, 17.732, 17.331, 17.381, 17.81, 17.13, 17.41, 17.334, 17.442, 17.464, 18.1, 17.212, 17.262, 17.55, 17.287, 17.545, 17.208, 17.438, 17.701, 17.548, 17.405, 17.131, 17.306, 17.127, 17.395, 17.708, 17.099, 17.713, 17.064, 17.433, 17.066, 16.962, 17.193, 17.613, 17.319]
m2_i_list= [267.61, 371.17, 8924.26, 239.58, 233.23, 274.62, 249.39, 372.31, 8994.59, 326.1, 310.14,253.81, 304.67, 261.06, 9136.38, 317.79, 236.46, 473.31, 463.55, 263.21, 308.23, 408.41, 486.24, 9029.74, 392.82, 415.19, 385.38, 295.87, 263.04, 9197.0, 273.77, 8915.21, 249.07, 420.69, 298.95, 268.5,281.85, 357.71, 303.78, 349.7]
m2_t_list= [6.934, 8.566, 8.9, 8.722, 8.351, 7.498, 7.559, 7.272, 7.492, 7.88, 7.22, 7.146, 7.182, 7.192, 7.272, 7.731, 7.348, 7.185, 7.459, 7.332, 7.427, 7.248, 7.211, 7.413, 7.699, 7.326, 7.313, 7.143, 7.122, 7.253, 7.73, 7.202, 7.747, 7.302, 7.303, 7.211, 7.243, 7.296, 7.429, 7.338]

In [13]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 1965.3732
  方差: 14469410.1671
  标准差: 3803.8678
imm_t_list:
  均值: 28.7643
  方差: 142.1278
  标准差: 11.9217
m1_i_list:
  均值: 1584.9008
  方差: 9697475.0543
  标准差: 3114.0769
m1_t_list:
  均值: 17.5906
  方差: 0.3348
  标准差: 0.5786
m2_i_list:
  均值: 1629.3597
  方差: 9678648.5029
  标准差: 3111.0526
m2_t_list:
  均值: 7.4799
  方差: 0.1894
  标准差: 0.4352


# len_topic=6

In [14]:
#soc_6
imm_i_list= [1054.16, 4131.06, 1233.8, 974.46, 839.37, 1159.43, 953.82, 1710.94, 1389.63, 973.75, 1796.43, 4196.84, 912.06, 851.36, 849.93, 6080.34, 1240.68, 831.36, 884.42, 4417.31]
imm_t_list= [606.975, 1387.121, 921.217, 719.641, 118.889, 677.96, 628.393, 1987.126, 1535.016, 194.879, 2235.833, 1418.934, 214.826, 169.104, 288.537, 3003.283, 1158.129, 126.632, 186.129, 1777.584]
m1_i_list= [882.94, 3321.87, 1017.07, 819.82, 697.84, 965.96, 808.77, 1403.06, 1131.18, 824.32, 1467.08, 3375.42, 785.47, 737.31, 728.22, 5445.64, 1037.59, 698.33, 784.42, 3599.89]
m1_t_list= [477.431, 487.247, 512.619, 533.271, 497.776, 516.919, 512.874, 528.286, 574.998, 510.017, 628.659, 644.102, 640.22, 624.119, 617.678, 604.542, 603.911, 496.778, 405.011, 632.022]
m2_i_list= [1044.28, 3949.7, 1192.73, 924.19, 790.3, 1133.11, 912.92, 1639.99, 1321.93, 961.36, 1728.91, 3981.29, 887.81, 841.52, 828.12, 5824.2, 1201.44, 810.21, 867.71, 4211.62]
m2_t_list= [40.088, 41.758, 43.394, 46.966, 44.401, 47.158, 45.791, 46.66, 44.885, 52.422, 53.645, 57.15, 55.551, 59.215, 59.051, 57.013, 54.963, 52.95, 54.311, 60.498]

In [15]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 1824.0575
  方差: 2274284.0260
  标准差: 1508.0730
imm_t_list:
  均值: 967.8104
  方差: 643962.5311
  标准差: 802.4728
m1_i_list:
  均值: 1526.6100
  方差: 1647122.4009
  标准差: 1283.4027
m1_t_list:
  均值: 552.4240
  方差: 4418.9098
  标准差: 66.4749
m2_i_list:
  均值: 1752.6670
  方差: 2058378.8871
  标准差: 1434.7052
m2_t_list:
  均值: 50.8935
  方差: 38.8375
  标准差: 6.2320


In [16]:
#twitter_6
imm_i_list= [3951.06, 4487.6, 4313.55, 17404.7, 4219.93, 3673.25, 16472.42, 15909.1, 4365.99, 17168.93]
imm_t_list= [1113.706, 1033.611, 1043.276, 5335.132, 1034.187, 1724.114, 5329.33, 5447.854, 1048.138, 5410.689]
m1_i_list= [3271.04, 3807.86, 3824.4, 15568.44, 3181.55, 3185.9, 15246.64, 15158.73, 3786.64, 15568.41]
m1_t_list= [1390.037, 1306.773, 1683.204, 2847.9, 1314.26, 1313.943, 2971.793, 2927.959, 1383.677, 3001.186]
m2_i_list= [3174.61, 3653.75, 3716.15, 16237.8, 3126.53, 3102.46, 15696.64, 15283.31, 3657.64, 16230.46]
m2_t_list= [196.8, 192.624, 195.126, 189.077, 193.603, 191.936, 190.404, 188.133, 190.164, 189.614]

In [17]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 9196.6530
  方差: 38107014.4811
  标准差: 6173.0879
imm_t_list:
  均值: 2852.0037
  方差: 4301863.1987
  标准差: 2074.0933
m1_i_list:
  均值: 8259.9610
  方差: 33916568.0772
  标准差: 5823.7933
m1_t_list:
  均值: 2014.0732
  方差: 579856.2643
  标准差: 761.4829
m2_i_list:
  均值: 8387.9350
  方差: 37350010.3654
  标准差: 6111.4655
m2_t_list:
  均值: 191.7481
  方差: 7.0249
  标准差: 2.6505


In [4]:
# twitter_6_2
imm_i_list= [3828.12, 3650.42, 5048.32, 15947.82, 3831.89, 3703.97, 4236.0, 3781.07, 4062.35, 4074.86]                                     
imm_t_list= [1083.39, 1658.203, 951.474, 5580.977, 1091.617, 1127.744, 1056.265, 1687.383, 1046.031, 1075.87]                                        
m1_i_list= [3402.78, 3228.56, 4095.02, 15284.13, 3293.54, 3339.14, 3827.54, 3347.15, 3292.62, 3683.69]                                             
m1_t_list= [1290.655, 1300.782, 1283.392, 2848.933, 1301.028, 1302.492, 1684.956, 1306.52, 1296.263, 1292.009]                                       
m2_i_list= [3181.62, 3118.86, 4001.87, 15423.25, 3151.25, 3204.96, 3726.96, 3206.96, 3165.69, 3466.95]                                               
m2_t_list= [184.915, 189.233, 186.253, 187.21, 189.574, 189.175, 188.421, 188.511, 190.431, 188.649]

In [5]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 5216.4820
  方差: 12943185.4542
  标准差: 3597.6639
imm_t_list:
  均值: 1635.8954
  方差: 1789250.8984
  标准差: 1337.6288
m1_i_list:
  均值: 4679.4170
  方差: 12566855.1186
  标准差: 3544.9760
m1_t_list:
  均值: 1490.7030
  方差: 218419.9583
  标准差: 467.3542
m2_i_list:
  均值: 4564.8370
  方差: 13178253.3144
  标准差: 3630.1864
m2_t_list:
  均值: 188.2372
  方差: 2.4778
  标准差: 1.5741


In [18]:
#wiki_6
imm_i_list= [984.17, 16728.23, 4098.07, 17791.08, 1631.83, 979.35, 841.76, 23876.82, 690.13, 500.7]
imm_t_list= [292.013, 9861.757, 3182.336, 11174.504, 348.886, 292.447, 294.533, 17433.604, 436.857, 466.149]
m1_i_list= [595.78, 16111.21, 3692.59, 17064.76, 774.44, 597.9, 512.25, 23275.52, 591.23, 214.92]
m1_t_list= [92.091, 87.561, 85.776, 84.604, 85.201, 84.429, 83.917, 82.777, 56.933, 60.224]
m2_i_list= [748.16, 16115.32, 3769.11, 17125.31, 1148.35, 737.51, 607.74, 23426.22, 594.78, 235.82]
m2_t_list= [41.037, 48.47, 43.836, 44.28, 43.201, 43.025, 42.099, 42.017, 42.649, 42.756]

In [19]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 6812.2140
  方差: 72521644.7683
  标准差: 8515.9641
imm_t_list:
  均值: 4378.3086
  方差: 34527314.6548
  标准差: 5875.9948
m1_i_list:
  均值: 6343.0600
  方差: 70578841.1178
  标准差: 8401.1214
m1_t_list:
  均值: 80.3513
  方差: 124.9584
  标准差: 11.1785
m2_i_list:
  均值: 6450.8320
  方差: 70304251.4393
  标准差: 8384.7631
m2_t_list:
  均值: 43.3370
  方差: 3.6978
  标准差: 1.9230


In [20]:
#gnu_6
imm_i_list= [74.85, 70.16, 71.3, 75.84, 73.08, 77.08, 73.8, 69.02, 83.25, 71.92, 74.64, 68.73, 72.77, 73.95, 68.83, 83.46, 66.2, 70.68, 83.25, 70.53, 70.3, 68.15, 68.63, 70.27, 72.91, 70.02, 75.79, 70.14, 80.72, 66.8, 71.9, 69.17, 74.22, 72.27, 75.94, 74.85, 72.91, 70.03, 71.46, 70.81]
imm_t_list= [4.896, 4.912, 5.023, 4.921, 4.934, 4.717, 4.753, 4.88, 4.689, 4.798, 4.68, 5.026, 4.674, 5.059, 5.02, 4.638, 4.985, 5.267, 4.85, 5.209, 5.015, 5.382, 5.348, 5.175, 5.129, 5.012, 4.941, 5.129, 4.671, 5.219, 5.097, 5.287, 4.965, 5.317, 4.83, 5.103, 5.197, 5.173, 5.203, 5.251]
m1_i_list= [68.32, 67.54, 69.17, 68.67, 65.9, 70.78, 67.87, 66.05, 71.13, 66.82, 65.56, 66.79, 65.41, 66.95, 65.01, 75.85, 68.01, 65.97, 71.04, 67.97, 67.36, 67.33, 66.7, 67.88, 65.65, 67.24, 71.28, 67.45, 70.43, 66.54, 66.51, 66.68, 67.15, 67.03, 67.68, 67.6, 66.57, 71.19, 68.41, 65.67]
m1_t_list= [0.473, 0.457, 0.516, 0.449, 0.463, 0.453, 0.434, 0.366, 0.41, 0.423, 0.451, 0.399, 0.377, 0.37, 0.418, 0.458, 0.361, 0.412, 0.391, 0.418, 0.473, 0.381, 0.422, 0.429, 0.435, 0.395, 0.437, 0.382, 0.388, 0.43, 0.384, 0.39, 0.404, 0.43, 0.437, 0.466, 0.434, 0.384, 0.401, 0.437]
m2_i_list= [65.71, 67.08, 67.76, 66.95, 64.78, 67.56, 65.42, 63.27, 69.43, 64.22, 64.58, 63.71, 63.19, 64.38, 62.8, 76.82, 65.29, 63.23, 71.02, 67.49, 63.75, 66.68, 65.49, 64.85, 64.26, 65.93, 67.37, 66.33, 68.65, 64.94, 63.62, 66.24, 64.93, 64.81, 64.06, 65.79, 64.25, 69.32, 66.5, 65.25]
m2_t_list= [0.412, 0.407, 0.444, 0.402, 0.409, 0.394, 0.385, 0.316, 0.359, 0.382, 0.381, 0.348, 0.329, 0.322, 0.355, 0.402, 0.314, 0.36, 0.338, 0.364, 0.418, 0.332, 0.362, 0.372, 0.38, 0.346, 0.386, 0.333, 0.338, 0.359, 0.338, 0.336, 0.353, 0.378, 0.386, 0.405, 0.379, 0.331, 0.343, 0.39]

In [21]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 72.7657
  方差: 17.4323
  标准差: 4.1752
imm_t_list:
  均值: 5.0094
  方差: 0.0425
  标准差: 0.2060
m1_i_list:
  均值: 67.8290
  方差: 4.4683
  标准差: 2.1138
m1_t_list:
  均值: 0.4210
  方差: 0.0012
  标准差: 0.0343
m2_i_list:
  均值: 65.9427
  方差: 6.4929
  标准差: 2.5481
m2_t_list:
  均值: 0.3672
  方差: 0.0010
  标准差: 0.0312


In [22]:
#flixster_6
imm_i_list= [286.89, 412.32, 630.47, 479.1, 293.45, 11837.2, 338.1, 454.28, 11901.9, 11327.43, 334.57, 427.81, 450.13, 432.76, 11410.52, 11169.72, 11895.12, 540.48, 281.88, 338.46, 313.14, 11656.98, 11883.0, 348.96, 577.73, 309.01, 494.74, 413.06, 598.14, 11110.71, 320.62, 10984.46, 348.14, 11172.65, 447.03, 600.49, 11080.84, 694.42, 325.67, 587.72]
imm_t_list= [46.207, 24.417, 22.057, 28.45, 44.707, 11.418, 28.766, 23.678, 10.69, 10.092, 29.259, 23.027, 26.081, 25.402, 10.846, 9.871, 10.931, 27.269, 45.523, 26.238, 29.361, 12.87, 11.403, 28.039, 27.633, 29.802, 27.409, 24.962, 28.095, 9.205, 27.449, 9.37, 29.126, 9.401, 23.23, 25.991, 9.918, 17.487, 29.319, 28.203]
m1_i_list= [225.41, 304.36, 473.9, 375.07, 213.92, 9730.78, 280.42, 312.19, 9838.73, 9500.97, 227.52, 302.76, 341.86, 305.5, 9397.6, 9168.65, 9654.69, 416.25, 227.7, 250.1, 245.22, 9510.0, 9819.2, 258.6, 449.47, 229.32, 380.19, 295.73, 426.43, 9154.05, 234.38, 9035.62, 258.87, 9074.73, 331.94, 435.48,9087.63, 503.35, 248.91, 424.93]
m1_t_list= [18.863, 19.428, 19.181, 18.776, 18.653, 18.826, 18.915, 18.632, 18.486, 19.211, 18.827, 18.561, 18.202, 18.663, 18.563, 18.742, 19.091, 18.254, 18.544, 18.485, 18.539, 18.622, 19.371, 18.867, 18.645, 18.497, 18.434, 18.623, 18.423, 18.807, 19.091, 18.454, 18.658, 18.479, 18.77, 18.39, 18.474, 18.505, 18.419, 18.306]
m2_i_list= [266.59, 329.72, 541.62, 452.27, 258.4, 9722.94, 328.14, 351.14, 9847.56, 9443.3, 288.97,328.75, 410.5, 351.7, 9417.01, 9257.7, 9694.57, 496.5, 268.33, 327.19, 289.08, 9553.08, 9822.63, 332.0, 542.16, 280.38, 462.84, 323.79, 488.03, 9163.3, 294.26, 9068.21, 338.89, 9020.08, 368.61, 499.2, 9231.81, 572.33, 312.36, 490.64]
m2_t_list= [9.248, 8.87, 9.321, 8.888, 8.679, 8.676, 9.104, 8.74, 8.338, 9.071, 9.045, 8.424, 8.483,8.638, 8.485, 8.833, 9.029, 8.414, 8.568, 8.383, 8.407, 8.555, 9.172, 8.919, 8.423, 8.643, 8.573, 8.758, 8.631, 8.658, 9.013, 8.373, 8.815, 8.595, 8.911, 8.294, 8.563, 8.528, 8.616, 8.524]

In [23]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 3737.7525
  方差: 25552941.5233
  标准差: 5054.9917
imm_t_list:
  均值: 23.0800
  方差: 97.5023
  标准差: 9.8743
m1_i_list:
  均值: 3048.8108
  方差: 17396396.7323
  标准差: 4170.8988
m1_t_list:
  均值: 18.6819
  方差: 0.0823
  标准差: 0.2869
m2_i_list:
  均值: 3095.9145
  方差: 17261525.1589
  标准差: 4154.6992
m2_t_list:
  均值: 8.7052
  方差: 0.0703
  标准差: 0.2651


# len_topic=8

In [24]:
#soc_8
imm_i_list= [1564.53, 1400.78, 1406.06, 4723.47, 862.39, 4592.01, 4380.52, 1940.87, 1496.92, 1355.28, 2233.37, 1682.31, 1294.19, 985.64, 1359.69, 1726.39, 2118.77, 957.21, 4338.47, 5588.47]
imm_t_list= [1613.529, 1496.53, 1162.121, 1785.593, 120.801, 1764.946, 1430.469, 1875.489, 1636.347,1388.917, 2116.034, 2238.806, 1163.259, 292.693, 1402.236, 2268.052, 1841.065, 208.186, 1404.894, 2584.534]
m1_i_list= [1324.63, 1086.49, 1167.41, 3913.37, 729.34, 3752.54, 3571.13, 1724.47, 1200.98, 1086.29,2023.02, 1411.9, 1092.97, 875.1, 1116.22, 1444.59, 1849.07, 809.15, 3511.93, 4965.84]
m1_t_list= [482.16, 498.701, 528.754, 529.959, 504.995, 544.467, 539.924, 536.899, 541.417, 527.573,527.73, 518.671, 517.328, 535.366, 528.586, 506.551, 535.428, 341.583, 526.855, 501.146]
m2_i_list= [1529.33, 1348.19, 1361.12, 4476.29, 830.13, 4367.14, 4177.45, 1909.45, 1440.98, 1283.64,2164.0, 1610.7, 1261.49, 971.58, 1314.42, 1663.8, 2061.22, 910.1, 4117.26, 5319.33]
m2_t_list= [46.743, 51.36, 56.675, 56.525, 55.969, 55.319, 55.395, 57.944, 57.116, 54.978, 54.48, 54.635, 54.652, 55.598, 55.794, 56.791, 55.596, 56.583, 56.821, 56.991]

In [25]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 2300.3670
  方差: 2124028.0555
  标准差: 1457.4046
imm_t_list:
  均值: 1489.7251
  方差: 421090.2085
  标准差: 648.9146
m1_i_list:
  均值: 1932.8220
  方差: 1515264.0977
  标准差: 1230.9606
m1_t_list:
  均值: 513.7047
  方差: 1816.1772
  标准差: 42.6166
m2_i_list:
  均值: 2205.8810
  方差: 1897142.7035
  标准差: 1377.3680
m2_t_list:
  均值: 55.2982
  方差: 5.6992
  标准差: 2.3873


In [2]:
#soc_8_2
imm_i_list= [4768.52, 1877.45, 5472.18, 1361.18, 1019.98, 2164.48, 4635.94, 1924.21, 1563.59, 4159.97, 1233.38, 1653.02, 975.1, 1399.18, 1869.89, 1512.78, 1219.67, 1232.17, 1348.77, 1119.13, 1258.01, 1077.34, 2229.96, 1251.97, 1664.89, 1795.09, 2525.92, 1048.97, 933.32, 1045.77, 1323.87, 1439.85, 816.69, 1324.07, 906.15, 4345.37, 959.76, 1878.3, 1893.37, 927.38]
imm_t_list= [1854.534, 2660.029, 2248.476, 960.425, 207.171, 1883.995, 1776.756, 2402.359, 1702.31, 1317.543, 986.329, 2125.406, 218.957, 1408.381, 2450.57, 1725.31, 830.379, 618.734, 1378.516, 772.362, 958.239, 556.436, 2046.878, 1182.394, 2361.398, 2631.423, 2334.791, 685.157, 123.786, 203.597, 890.579, 1541.604, 117.722, 1335.539, 277.767, 1380.215, 179.49, 2645.266, 2235.341, 180.484]
m1_i_list= [3974.0, 1675.11, 4910.7, 1088.59, 882.96, 1915.33, 3761.81, 1650.85, 1328.8, 3331.09, 951.78, 1384.93, 813.76, 1139.87, 1576.62, 1230.51, 996.21, 1028.85, 1043.48, 876.95, 949.38, 892.52, 2019.35, 977.3, 1387.95, 1547.73,2309.91, 845.63, 780.31, 907.37, 1080.58, 1149.8, 721.24, 1112.43, 768.67, 3517.56, 821.66, 1604.69, 1578.0, 791.18]
m1_t_list= [496.253, 495.641, 496.07, 501.191, 468.627, 501.1, 508.029, 508.598, 504.271, 510.994, 508.112, 498.538,481.884, 497.51, 506.548, 495.358, 502.256, 496.722, 498.02, 628.876, 668.417, 662.971, 679.33, 685.035, 681.091, 685.197, 669.311, 534.805, 323.673, 350.652, 520.232, 520.67, 481.352, 504.543, 516.022, 503.594, 502.821, 512.925, 518.151, 496.507]
m2_i_list= [4548.03, 1837.75, 5283.66, 1307.91, 978.98, 2114.42, 4367.07, 1858.34, 1510.25, 3990.34, 1167.17, 1594.57, 938.49, 1341.75, 1789.73, 1443.24, 1167.05, 1162.61, 1297.33, 1065.12, 1171.14, 1042.64, 2161.07, 1217.81, 1595.01, 1734.97, 2465.82, 1016.38, 850.14, 982.12, 1273.47, 1392.14, 806.64, 1284.69, 880.45, 4143.79, 929.29, 1783.07, 1814.8, 871.07]
m2_t_list= [54.874, 52.894, 53.445, 54.338, 55.078, 54.524, 54.279, 53.653, 55.562, 53.755, 54.202, 53.753, 52.846, 52.539, 54.19, 54.451, 54.05, 53.549, 52.529, 52.851, 66.93, 67.389, 72.976, 69.316, 70.938, 70.41, 68.407, 72.702, 61.276, 63.372, 56.215, 56.68, 57.494, 52.878, 56.441, 53.773, 55.915, 54.36, 54.828, 56.44]

lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 1828.9160
  方差: 1338119.7692
  标准差: 1156.7713
imm_t_list:
  均值: 1334.9162
  方差: 668555.9322
  标准差: 817.6527
m1_i_list:
  均值: 1533.1365
  方差: 971809.0325
  标准差: 985.8037
m1_t_list:
  均值: 528.0474
  方差: 6515.4977
  标准差: 80.7186
m2_i_list:
  均值: 1754.5080
  方差: 1224687.0868
  标准差: 1106.6558
m2_t_list:
  均值: 57.9026
  方差: 41.0563
  标准差: 6.4075


In [26]:
#twitter_8
imm_i_list= [3887.53, 4026.93, 4328.09, 3745.37, 4121.45, 4718.81, 3963.61, 4033.56, 16330.13, 3995.5]
imm_t_list= [1114.997, 1078.078, 1059.344, 1161.416, 1069.419, 1008.425, 1075.898, 1055.312, 5460.668, 1075.195]
m1_i_list= [3270.3, 3382.73, 3824.99, 3301.59, 3353.88, 3844.44, 3213.59, 3346.83, 15259.14, 3310.76]
m1_t_list= [1361.292, 1354.791, 1713.911, 1397.272, 1441.155, 1384.443, 1371.533, 1359.124, 2941.986, 1354.77]
m2_i_list= [3147.17, 3279.72, 3703.05, 3185.83, 3220.49, 3700.44, 3118.24, 3176.8, 15393.13, 3156.83]
m2_t_list= [255.892, 249.8, 248.497, 248.914, 251.746, 253.709, 252.671, 248.142, 251.548, 249.419]

In [27]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 5315.0980
  方差: 13545704.1206
  标准差: 3680.4489
imm_t_list:
  均值: 1515.8752
  方差: 1730454.8296
  标准差: 1315.4675
m1_i_list:
  均值: 4610.8250
  方差: 12643084.4587
  标准差: 3555.7115
m1_t_list:
  均值: 1568.0277
  方差: 220394.0190
  标准差: 469.4614
m2_i_list:
  均值: 4508.1700
  方差: 13208168.4707
  标准差: 3634.3044
m2_t_list:
  均值: 251.0338
  方差: 5.7632
  标准差: 2.4007


In [6]:
# twitter_8_2
imm_i_list= [4360.68, 4333.94, 4355.2, 16374.92, 5415.61, 16986.47, 5587.79, 4389.88, 4521.94, 4953.5]
imm_t_list= [976.785, 960.801, 970.023, 5415.393, 970.055, 5255.876, 1007.968, 1022.254, 986.346, 923.441]
m1_i_list= [3733.19, 3306.46, 3706.47, 15173.85, 4736.92, 15167.66, 4763.04, 3971.67, 3895.07, 4437.38]
m1_t_list= [1422.47, 1374.233, 1363.925, 3030.331, 1760.629, 3039.803, 1728.296, 1770.06, 1399.281, 685.762]
m2_i_list= [3564.28, 3126.55, 3475.91, 15308.3, 4554.69, 15933.1, 4617.12, 3812.53, 3768.85, 4197.27]
m2_t_list= [247.406, 250.559, 251.391, 251.697, 251.857, 248.719, 250.651, 250.737, 251.431, 250.172]

In [7]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 7127.9930
  方差: 23017003.7797
  标准差: 4797.6040
imm_t_list:
  均值: 1848.8942
  方差: 3041243.9507
  标准差: 1743.9163
m1_i_list:
  均值: 6289.1710
  方差: 19913525.5480
  标准差: 4462.4573
m1_t_list:
  均值: 1757.4790
  方差: 495640.3319
  标准差: 704.0173
m2_i_list:
  均值: 6235.8600
  方差: 22232927.1120
  标准差: 4715.1805
m2_t_list:
  均值: 250.4620
  方差: 1.7855
  标准差: 1.3362


In [28]:
#wiki_8
imm_i_list= [18613.39, 19509.85, 2336.08, 19818.77, 2162.46, 2733.46, 10408.77, 4435.66, 3079.37, 28883.61]
imm_t_list= [13157.826, 15105.66, 1508.255, 12793.717, 1788.694, 2156.725, 6911.756, 3119.572, 1036.785, 15736.83]
m1_i_list= [17788.06, 18945.98, 1940.58, 18970.34, 1660.31, 2162.89, 10094.96, 3957.02, 2044.3, 28345.8]
m1_t_list= [93.538, 95.654, 98.535, 100.359, 107.684, 104.193, 103.587, 96.204, 95.196, 89.651]
m2_i_list= [17893.72, 18997.84, 1988.18, 19176.78, 1822.64, 2281.74, 10142.89, 4156.76, 2562.68, 28564.45]
m2_t_list= [55.053, 53.785, 54.682, 54.801, 59.81, 64.009, 62.636, 55.029, 54.394, 50.072]

In [29]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 11198.1420
  方差: 85526066.1473
  标准差: 9248.0304
imm_t_list:
  均值: 7331.5820
  方差: 34382016.1679
  标准差: 5863.6180
m1_i_list:
  均值: 10591.0240
  方差: 84997148.0750
  标准差: 9219.3898
m1_t_list:
  均值: 98.4601
  方差: 27.3289
  标准差: 5.2277
m2_i_list:
  均值: 10758.7680
  方差: 84647436.0038
  标准差: 9200.4041
m2_t_list:
  均值: 56.4271
  方差: 16.8514
  标准差: 4.1050


In [30]:
#gnu_8
imm_i_list= [70.64, 75.68, 68.71, 79.93, 79.24, 76.51, 71.98, 72.29, 74.95, 72.47, 75.87, 75.9, 73.57, 75.33, 76.16, 74.42, 76.36, 75.61, 76.45, 74.02, 71.84, 75.13, 78.71, 78.53, 78.84, 80.36, 78.7, 71.27, 79.58, 73.29, 75.1, 77.85, 79.63, 77.76, 80.23, 80.09, 80.57, 79.26, 75.45, 75.23]
imm_t_list= [6.005, 5.111, 5.348, 5.878, 5.05, 4.651, 4.855, 5.018, 4.525, 5.018, 4.706, 4.972, 4.718, 4.751, 4.992, 5.202, 4.782, 5.211, 5.034, 5.417, 5.914, 5.613, 5.007, 5.688, 5.236, 4.884, 4.924, 5.163, 4.856, 5.052, 4.943, 4.798, 5.034, 5.119, 4.91, 4.918, 4.976, 4.946, 5.154, 5.131]
m1_i_list= [69.87, 67.58, 67.31, 70.48, 72.66, 69.45, 68.7, 67.54, 71.39, 67.57, 69.36, 70.27, 69.15, 69.89, 70.27, 70.25, 70.8, 70.03, 69.2, 66.43, 67.21, 68.18, 69.26, 72.4, 70.83, 70.81, 70.97, 67.48, 74.69, 68.9, 67.7, 70.76, 70.51, 68.48, 69.97, 70.61, 72.75, 71.31, 68.33, 68.09]
m1_t_list= [0.875, 0.637, 0.646, 0.689, 0.759, 1.0, 0.648, 0.565, 0.504, 0.56, 0.451, 0.531, 0.498, 0.453, 0.444, 0.551, 0.504, 0.459, 0.506, 0.471, 0.472, 0.92, 0.818, 0.843, 0.86, 0.671, 0.6, 0.576, 0.51, 0.534, 0.539, 0.436, 0.488, 0.478, 0.51, 0.43, 0.45, 0.441, 0.475, 0.527]
m2_i_list= [65.0, 66.65, 66.91, 66.73, 72.73, 69.62, 66.09, 64.86, 71.25, 64.96, 64.95, 67.64, 66.19, 68.48, 67.79, 69.72, 67.98, 68.91, 66.17, 63.8, 64.53, 68.86, 65.61, 67.62, 67.48, 68.85, 70.46, 64.49, 69.7, 64.7, 64.1, 70.85, 67.64, 64.37, 67.56, 66.9, 69.72, 68.51, 66.02, 64.59]
m2_t_list= [0.79, 0.57, 0.571, 0.549, 0.704, 0.95, 0.599, 0.515, 0.445, 0.512, 0.402, 0.479, 0.426, 0.406, 0.396, 0.497, 0.447, 0.406, 0.431, 0.421, 0.389, 0.87, 0.734, 0.74, 0.789, 0.618, 0.549, 0.494, 0.467, 0.479, 0.482, 0.38, 0.432, 0.425, 0.435, 0.377, 0.392, 0.386, 0.422, 0.461]

In [31]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 76.0877
  方差: 9.0227
  标准差: 3.0038
imm_t_list:
  均值: 5.0878
  方差: 0.1111
  标准差: 0.3333
m1_i_list:
  均值: 69.6860
  方差: 3.0376
  标准差: 1.7429
m1_t_list:
  均值: 0.5832
  方差: 0.0223
  标准差: 0.1492
m2_i_list:
  均值: 67.2248
  方差: 4.8560
  标准差: 2.2036
m2_t_list:
  均值: 0.5209
  方差: 0.0208
  标准差: 0.1444


In [32]:
#flixster_8
imm_i_list= [334.1, 11617.45, 686.46, 12042.22, 454.62, 11938.62, 608.15, 479.89, 11228.15, 331.91, 710.13, 360.3, 544.83, 537.51, 12018.24, 11610.8, 12102.78, 564.99, 12270.81, 530.79, 366.43, 11591.08, 473.72, 11381.04, 477.81, 607.88, 732.51, 11231.97, 377.46, 11742.05, 489.54, 446.78, 681.23, 346.96, 687.56, 11303.21, 499.94, 605.01, 11510.97, 666.1]
imm_t_list= [30.746, 11.344, 23.562, 13.306, 28.693, 12.279, 32.144, 27.853, 9.727, 29.596, 17.547, 27.91, 26.242, 24.959, 11.567, 10.875, 11.86, 29.462, 11.361, 29.081, 25.047, 9.575, 23.572, 10.702, 24.075, 19.813, 17.293, 11.028, 25.175, 10.657, 24.463, 26.183, 20.234, 29.76, 17.982, 10.681, 27.614, 33.162, 10.463, 17.257]
m1_i_list= [251.46, 9538.91, 489.04, 9927.23, 332.73, 9839.14, 460.67, 369.35, 9175.86, 255.35, 527.27, 272.85, 410.66, 352.98, 9765.62, 9573.17, 9991.38, 435.05, 10068.51, 396.15, 272.98, 9409.25, 329.6, 9453.21, 335.51, 430.7, 542.86, 9417.0, 294.48, 9535.99, 347.21, 334.52, 502.4, 272.31, 518.33, 9300.23, 343.42, 463.01, 9456.89, 476.98]
m1_t_list= [20.486, 21.123, 22.151, 22.25, 21.388, 21.84, 20.877, 20.167, 20.698, 20.27, 20.158, 20.009, 20.389, 20.142, 20.26, 20.205, 20.759, 20.358, 20.208, 20.272, 20.182, 20.288, 19.816, 20.263, 19.88, 20.141, 20.244, 20.311, 19.957, 20.59, 20.139, 19.902, 20.072, 19.755, 20.172, 20.276, 20.473, 21.71, 21.468, 20.912]
m2_i_list= [304.24, 9619.55, 560.2, 9924.42, 373.29, 9831.91, 551.04, 444.45, 9208.67, 311.08, 618.13, 357.71, 491.16, 423.69, 9770.38, 9562.76, 10024.83, 503.83, 9991.9, 469.64, 360.67, 9478.12, 376.26, 9448.41, 380.6, 504.81, 618.03, 9401.07, 372.24, 9562.24, 383.51, 363.41, 567.95, 324.36, 597.99, 9367.45, 425.78, 555.34, 9548.02, 547.86]
m2_t_list= [9.866, 11.149, 11.874, 12.241, 11.582, 11.724, 10.909, 10.443, 10.216, 10.68, 10.158, 10.043, 10.248, 9.967, 10.22, 10.074, 10.669, 10.587, 10.347, 10.675, 10.148, 10.363, 9.942, 10.211, 9.956, 10.299, 10.191, 10.343, 10.099, 10.554, 10.216, 10.178, 10.32, 10.199, 10.241, 10.292, 10.684, 10.249, 11.465, 10.855]

In [33]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 4429.8000
  方差: 28391182.8403
  标准差: 5328.3377
imm_t_list:
  均值: 20.3712
  方差: 60.2427
  标准差: 7.7616
m1_i_list:
  均值: 3611.7565
  方差: 19361951.5623
  标准差: 4400.2218
m1_t_list:
  均值: 20.5140
  方差: 0.3877
  标准差: 0.6227
m2_i_list:
  均值: 3663.1750
  方差: 19160413.2102
  标准差: 4377.2609
m2_t_list:
  均值: 10.5119
  方差: 0.3110
  标准差: 0.5577


# len_topic=10

In [34]:
#soc_10
imm_i_list= [1094.61, 1262.4, 4723.55, 1092.84, 1566.39, 1126.71, 1088.51, 5887.65, 5890.52, 1295.62, 1725.27, 1640.28, 2452.5, 1629.42, 1804.99, 1065.78, 4482.36, 1085.36, 1146.77, 1658.74]
imm_t_list= [202.06, 1128.286, 1984.456, 809.001, 1539.598, 226.515, 394.542, 2673.939, 2720.653, 1114.336, 2311.646, 2134.451, 2256.923, 1601.395, 2740.887, 218.739, 1596.705, 264.162, 842.06, 2346.349]
m1_i_list= [922.18, 932.95, 3854.34, 888.82, 1344.48, 964.79, 917.11, 5186.86, 5188.37, 1077.2, 1377.22, 1321.58, 2188.33, 1408.36, 1546.58, 909.66, 3688.05, 926.4, 952.37, 1423.75]
m1_t_list= [501.401, 505.935, 526.862, 525.886, 511.231, 490.32, 531.709, 523.041, 498.461, 519.975,513.975, 513.738, 517.017, 521.04, 514.242, 495.339, 518.147, 511.488, 528.604, 539.165]
m2_i_list= [1040.49, 1208.77, 4460.8, 1067.73, 1507.42, 1072.64, 1072.28, 5605.89, 5647.45, 1250.33,1599.84, 1566.11, 2370.38, 1578.0, 1738.34, 1035.79, 4279.61, 1068.16, 1110.73, 1603.43]
m2_t_list= [66.153, 65.137, 68.934, 66.116, 67.594, 65.624, 67.123, 68.708, 67.936, 68.463, 66.215, 66.207, 68.965, 68.208, 69.285, 67.08, 66.253, 69.8, 71.619, 67.329]

In [35]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 2186.0135
  方差: 2537045.4478
  标准差: 1592.8105
imm_t_list:
  均值: 1455.3352
  方差: 793455.5271
  标准差: 890.7612
m1_i_list:
  均值: 1850.9700
  方差: 1921825.7773
  标准差: 1386.2993
m1_t_list:
  均值: 515.3788
  方差: 150.5235
  标准差: 12.2688
m2_i_list:
  均值: 2094.2095
  方差: 2290016.6147
  标准差: 1513.2801
m2_t_list:
  均值: 67.6374
  方差: 2.5206
  标准差: 1.5877


In [3]:
#soc_10_2
imm_i_list= [2212.61, 1681.66, 1554.91, 1236.75, 1751.01, 1719.93, 1366.5, 1888.74, 1885.52, 4788.52, 1771.04, 1473.84, 2245.27, 2309.16, 1434.9, 2070.32, 2025.33, 1053.28, 4798.13, 1569.02, 1731.2, 4334.7, 1547.9, 1043.24, 2607.27, 964.61, 1461.82, 1126.97, 1514.36, 1872.45, 1958.02, 1894.53, 1642.28, 1037.69, 1100.49, 1467.11, 1917.71, 1965.36, 6724.18, 1207.56]
imm_t_list= [1963.733, 2072.109, 1886.776, 819.636, 2244.256, 2131.598, 1389.751, 2738.726, 2634.098, 1770.211, 2482.626, 1620.755, 1982.972, 1980.22, 1450.938, 1792.345, 1766.755, 299.111, 1812.528, 1907.379, 2335.245, 1428.016, 1378.542, 191.601, 2235.852, 199.715, 1496.276, 350.85, 1463.172, 2501.325, 1908.453, 2804.895, 1932.382, 146.657, 279.317, 1285.111, 2846.459, 1549.955, 3344.105, 803.019]
m1_i_list= [1949.75, 1377.26, 1320.68, 1007.17, 1448.69, 1460.25, 1095.68, 1654.73, 1631.16, 3906.47, 1496.67, 1209.04, 2035.07, 2062.78, 1156.65, 1817.1, 1754.63, 925.55, 3962.71, 1359.97, 1463.55, 3506.18, 1296.89, 888.57, 2379.07,802.98, 1207.41, 909.4, 1291.25, 1604.6, 1731.39, 1641.08, 1394.87, 872.34, 904.8, 1202.78, 1656.69, 1670.24, 6406.83, 984.65]
m1_t_list= [496.004, 507.675, 504.279, 504.233, 510.873, 509.478, 511.408, 511.068, 519.354, 497.747, 506.677, 508.753, 512.898, 488.253, 504.495, 522.457, 579.597, 571.331, 574.593, 566.547, 552.821, 557.04, 572.388, 548.053, 519.897, 512.158, 520.214, 497.039, 510.024, 509.079, 507.54, 515.462, 515.22, 484.369, 515.597, 514.421, 498.052, 514.424,520.501, 522.768]
m2_i_list= [2163.23, 1602.37, 1499.18, 1196.52, 1686.82, 1653.1, 1285.41, 1834.16, 1802.88, 4542.15, 1706.74, 1420.72, 2195.11, 2240.39, 1334.89, 2014.18, 1960.37, 1032.97, 4613.29, 1531.59, 1667.72, 4113.25, 1487.18, 1042.28, 2518.14, 918.53, 1396.07, 1097.02, 1469.41, 1809.22, 1901.62, 1829.63, 1576.72, 976.8, 1066.8, 1403.44, 1823.24, 1900.57, 6547.51, 1160.91]
m2_t_list= [58.252, 72.271, 66.372, 65.253, 64.406, 66.94, 66.649, 65.695, 68.483, 64.776, 67.901, 68.423, 66.37, 69.627, 65.72, 65.735, 68.903, 69.729, 69.866, 70.364, 72.195, 70.863, 71.1, 69.902, 69.14, 68.1, 69.541, 68.791, 65.081, 67.096, 67.535, 65.498, 68.092, 67.763, 66.148, 67.803, 63.912, 71.075, 66.598, 67.298]
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 1998.8972
  方差: 1337327.8494
  标准差: 1156.4289
imm_t_list:
  均值: 1680.6868
  方差: 628222.5998
  标准差: 792.6049
m1_i_list:
  均值: 1711.1895
  方差: 1089991.4080
  标准差: 1044.0265
m1_t_list:
  均值: 520.3697
  方差: 595.7841
  标准差: 24.4087
m2_i_list:
  均值: 1925.5533
  方差: 1241798.4426
  标准差: 1114.3601
m2_t_list:
  均值: 67.6316
  方差: 6.7461
  标准差: 2.5973


In [36]:
#twitter_10
imm_i_list= [5117.72, 4333.87, 4766.94, 4546.39, 17122.26, 4938.21, 16999.46, 16418.05, 4609.8, 4621.38]
imm_t_list= [980.98, 1018.984, 1023.582, 972.051, 5638.88, 1012.25, 5349.388, 5540.445, 1045.611, 951.11]
m1_i_list= [4398.3, 3628.23, 3898.02, 3693.74, 15554.01, 3921.21, 15210.0, 15305.81, 3791.09, 3731.23]
m1_t_list= [1834.869, 1469.707, 1555.623, 1523.979, 3262.872, 1547.326, 3242.786, 3200.777, 1555.514, 1427.411]
m2_i_list= [4156.29, 3495.11, 3743.83, 3539.43, 15866.77, 3745.47, 15958.81, 15451.63, 3687.28, 3505.32]
m2_t_list= [352.93, 325.798, 324.471, 328.314, 324.895, 328.656, 325.389, 324.28, 329.326, 325.862]

In [37]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 8347.4080
  方差: 31027378.3737
  标准差: 5570.2225
imm_t_list:
  均值: 2353.3281
  方差: 4274373.5032
  标准差: 2067.4558
m1_i_list:
  均值: 7313.1640
  方差: 27773316.9423
  标准差: 5270.0396
m1_t_list:
  均值: 2062.0864
  方差: 600556.6464
  标准差: 774.9559
m2_i_list:
  均值: 7314.9940
  方差: 30604572.0197
  标准差: 5532.1399
m2_t_list:
  均值: 328.9921
  方差: 66.6120
  标准差: 8.1616


In [8]:
#twitter_10_2
imm_i_list= [5171.37, 16983.53, 4957.49, 4243.89, 16267.56, 5650.88, 4608.55, 17422.84, 16388.01, 16899.38]
imm_t_list= [942.158, 5414.851, 918.745, 1020.569, 5257.676, 992.653, 977.276, 5287.89, 5232.926, 5073.747]
m1_i_list= [4468.41, 15607.6, 4232.32, 3685.07, 15283.64, 4798.35, 3748.92, 15641.2, 15295.65, 15238.76]
m1_t_list= [1805.118, 3161.106, 1436.893, 1434.553, 3127.904, 1800.117, 1418.779, 3083.175, 3175.634, 3154.29]
m2_i_list= [4179.74, 15948.76, 4034.89, 3511.04, 15405.55, 4598.35, 3644.06, 16327.9, 15471.01, 16052.06]
m2_t_list= [320.557, 323.154, 325.508, 322.227, 326.088, 323.265, 324.708, 322.833, 327.411, 326.859]

In [9]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 10859.3500
  方差: 35403180.0186
  标准差: 5950.0571
imm_t_list:
  均值: 3111.8491
  方差: 4592966.7369
  标准差: 2143.1208
m1_i_list:
  均值: 9799.9920
  方差: 31614977.5413
  标准差: 5622.7198
m1_t_list:
  均值: 2359.7569
  方差: 626642.2954
  标准差: 791.6074
m2_i_list:
  均值: 9917.3360
  方差: 35228144.3712
  标准差: 5935.3302
m2_t_list:
  均值: 324.2610
  方差: 4.3878
  标准差: 2.0947


In [38]:
#wiki_10
imm_i_list= [1725.39, 24212.62, 29855.41, 1260.62, 25310.63, 22234.31, 2185.0, 28241.93, 1526.24, 29224.07]
imm_t_list= [219.789, 21317.294, 14428.814, 268.898, 18158.549, 14521.357, 213.745, 14484.088, 314.83, 13658.783]
m1_i_list= [975.29, 23471.96, 29205.12, 679.57, 24453.85, 21178.8, 1182.92, 27644.54, 745.04, 28475.04]
m1_t_list= [102.466, 104.175, 99.09, 103.237, 103.102, 99.494, 101.997, 100.405, 102.196, 94.889]
m2_i_list= [1165.28, 23527.99, 29420.05, 954.07, 24758.34, 21535.57, 1358.11, 27842.5, 917.22, 28798.06]
m2_t_list= [63.56, 61.584, 63.26, 62.121, 63.073, 58.712, 61.093, 60.464, 61.261, 60.07]

In [39]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 16577.6220
  方差: 152773613.8421
  标准差: 12360.1624
imm_t_list:
  均值: 9758.6147
  方差: 64753137.5631
  标准差: 8046.9334
m1_i_list:
  均值: 15801.2130
  方差: 153203685.2115
  标准差: 12377.5476
m1_t_list:
  均值: 101.1051
  方差: 6.7581
  标准差: 2.5996
m2_i_list:
  均值: 16027.7190
  方差: 153647101.5151
  标准差: 12395.4468
m2_t_list:
  均值: 61.5198
  方差: 2.1318
  标准差: 1.4601


In [40]:
#gnu_10
imm_i_list= [78.39, 78.4, 77.1, 83.15, 79.57, 76.66, 79.06, 87.11, 78.84, 76.83, 76.99, 80.94, 79.43, 81.24, 75.53, 83.39, 74.05, 82.3, 75.91, 78.55, 77.22, 79.03, 75.49, 83.31, 80.35, 74.15, 77.62, 79.16, 79.22, 80.43, 78.73, 80.81, 83.22, 84.29, 79.26, 78.0, 77.72, 79.6, 79.89, 83.59]
imm_t_list= [4.693, 4.651, 4.521, 4.602, 4.38, 4.561, 5.188, 5.127, 4.876, 4.768, 4.775, 4.563, 4.468, 4.423, 4.765, 4.452, 4.826, 4.515, 4.542, 4.675, 4.78, 4.745, 4.857, 4.717, 4.848, 5.262, 4.926, 5.196, 5.158, 4.911, 4.998, 5.015, 4.805, 4.941, 4.82, 4.808, 4.83, 4.808, 4.83, 4.48]
m1_i_list= [68.86, 69.52, 67.89, 71.77, 71.43, 71.24, 70.87, 74.44, 68.69, 71.97, 69.99, 72.41, 73.13, 73.07, 71.3, 72.45, 68.92, 74.8, 68.91, 71.95, 70.53, 69.05, 68.38, 74.01, 69.33, 69.5, 71.66, 70.21, 69.65, 71.97, 69.6, 71.91, 74.17, 71.03, 69.81, 69.06, 72.96, 71.92, 70.27, 74.75]
m1_t_list= [0.545, 0.504, 0.552, 0.491, 0.474, 0.555, 0.579, 0.767, 0.762, 0.685, 0.646, 0.595, 0.479, 0.565, 0.508, 0.473, 0.529, 0.497, 0.489, 0.48, 0.477, 0.485, 0.537, 0.496, 0.456, 0.488, 0.534, 0.513, 0.637, 0.592, 0.595, 0.518, 0.573, 0.554, 0.473, 0.561, 0.515, 0.463, 0.484, 0.499]
m2_i_list= [66.88, 66.71, 66.91, 68.25, 70.4, 66.78, 69.0, 68.48, 65.66, 70.65, 65.85, 70.46, 70.95,71.69, 67.07, 66.47, 67.78, 73.73, 66.93, 70.25, 70.5, 65.33, 64.04, 69.89, 65.72, 65.78, 68.69, 66.56, 69.38, 71.01, 66.63, 69.18, 68.23, 72.37, 68.49, 67.04, 71.24, 69.19, 68.38, 71.72]
m2_t_list= [0.482, 0.456, 0.483, 0.437, 0.423, 0.48, 0.504, 0.668, 0.69, 0.617, 0.59, 0.548, 0.43, 0.5, 0.458, 0.424, 0.467, 0.428, 0.439, 0.43, 0.428, 0.43, 0.46, 0.424, 0.401, 0.429, 0.492, 0.39, 0.581, 0.539, 0.515, 0.461, 0.501, 0.495, 0.418, 0.5, 0.463, 0.408, 0.433, 0.449]

In [41]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 79.3632
  方差: 7.9890
  标准差: 2.8265
imm_t_list:
  均值: 4.7776
  方差: 0.0494
  标准差: 0.2223
m1_i_list:
  均值: 71.0845
  方差: 3.4822
  标准差: 1.8661
m1_t_list:
  均值: 0.5406
  方差: 0.0054
  标准差: 0.0736
m2_i_list:
  均值: 68.5067
  方差: 4.8978
  标准差: 2.2131
m2_t_list:
  均值: 0.4793
  方差: 0.0047
  标准差: 0.0688


In [42]:
#flixster_10
imm_i_list= [527.87, 12390.9, 12315.75, 584.17, 11382.49, 565.34, 517.54, 12457.72, 12578.5, 654.65,11318.86, 523.95, 11775.77, 504.2, 11762.28, 12238.0, 612.23, 12186.62, 11728.32, 12365.71, 544.36, 713.4, 11662.28, 12416.79, 11588.38, 12567.86, 11496.31, 718.82, 544.57, 12231.11, 12464.8, 11654.81,574.9, 576.86, 12351.14, 556.12, 12409.19, 12362.52, 12266.86, 11673.62]
imm_t_list= [27.62, 13.791, 13.406, 30.438, 10.792, 27.698, 26.732, 11.818, 13.025, 19.013, 10.141, 26.621, 11.116, 26.529, 10.362, 11.828, 20.76, 12.421, 10.518, 12.683, 28.045, 16.981, 10.347, 11.712, 9.65, 13.314, 10.996, 16.619, 27.418, 12.148, 12.402, 10.234, 30.586, 31.133, 12.302, 27.637, 12.288, 12.759, 12.485, 10.754]
m1_i_list= [365.16, 10286.06, 10258.45, 435.99, 9331.16, 430.39, 358.01, 10206.03, 10261.46, 471.02,9369.95, 354.96, 9698.06, 367.08, 9663.92, 10137.15, 462.81, 10048.57, 9630.99, 10393.32, 362.8, 523.9, 9606.17, 10124.93, 9638.27, 10229.56, 9471.24, 509.02, 372.12, 10050.74, 10204.89, 9609.66, 429.97, 438.96, 10146.6, 370.66, 10257.14, 10368.42, 10095.89, 9714.98]
m1_t_list= [21.711, 22.73, 23.556, 22.773, 22.556, 22.031, 21.976, 23.238, 22.64, 22.393, 21.945, 22.346, 22.247, 22.353, 22.064, 22.558, 22.295, 22.189, 22.502, 22.605, 22.344, 22.149, 21.816, 22.625,22.754, 22.484, 22.874, 21.606, 21.755, 21.985, 22.584, 23.006, 24.033, 23.029, 22.587, 22.653, 22.158, 22.544, 22.834, 22.575]
m2_i_list= [436.65, 10228.33, 10169.18, 516.28, 9406.9, 532.43, 422.31, 10177.72, 10202.43, 544.63, 9320.81, 420.7, 9625.64, 421.54, 9663.06, 10129.23, 547.12, 10003.57, 9590.22, 10344.38, 452.4, 590.36, 9719.47, 10032.85, 9597.67, 10223.83, 9402.28, 602.35, 430.58, 10016.67, 10187.15, 9586.23, 526.43, 522.52, 10179.99, 470.9, 10253.36, 10308.96, 10046.14, 9636.82]
m2_t_list= [11.73, 12.836, 13.539, 12.741, 12.757, 11.887, 12.269, 13.222, 12.688, 12.585, 12.142, 12.52, 12.23, 12.452, 12.171, 12.605, 12.474, 12.114, 12.674, 12.618, 12.589, 12.461, 11.979, 12.698, 12.795, 12.708, 12.712, 11.948, 12.172, 12.044, 12.705, 12.882, 12.617, 13.03, 12.507, 12.795, 12.067, 12.681, 12.784, 12.669]

In [43]:
lists_to_calculate = {
    "imm_i_list": imm_i_list,
    "imm_t_list": imm_t_list,
    "m1_i_list": m1_i_list,
    "m1_t_list": m1_t_list,
    "m2_i_list": m2_i_list,
    "m2_t_list": m2_t_list
}

for name, lst in lists_to_calculate.items():
    mean, variance = calculate_mean_and_variance(lst)
    print(f"{name}:")
    print(f"  均值: {mean:.4f}")
    print(f"  方差: {variance:.4f}")
    print(f"  标准差: {math.sqrt(variance):.4f}")  # 额外计算标准差作为参考

imm_i_list:
  均值: 7759.1392
  方差: 31013716.8607
  标准差: 5568.9960
imm_t_list:
  均值: 16.9281
  方差: 53.9630
  标准差: 7.3460
m1_i_list:
  均值: 6376.4115
  方差: 21378598.7270
  标准差: 4623.6997
m1_t_list:
  均值: 22.4776
  方差: 0.2324
  标准差: 0.4820
m2_i_list:
  均值: 6387.2522
  方差: 20890650.8406
  标准差: 4570.6292
m2_t_list:
  均值: 12.5274
  方差: 0.1351
  标准差: 0.3676
